\section*{Channels of interstate risk sharing: United States 1963-1990 by Asrdubali et al}

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
import requests, zipfile, io # So that we can download and unzip files\n",
from matplotlib2tikz import save as tikz_save
import matplotlib
import statsmodels.tsa.filters as filters
import quandl as Quandl
import sqlite3

State income variables

In [2]:
r = requests.get('http://apps.bea.gov/regional/zip/CA4.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
f = z.open('CA4_1969_2016__ALL_AREAS.csv')
CA4_table = pd.read_csv(f,encoding = "ISO-8859-1", low_memory=False)

Gross State Products

In [3]:
keep = np.arange(6,54)
keep = keep.tolist()
keep.append(1)
data = CA4_table.iloc[:,keep]
data = data[0:73554] #some useless columns
data = data.drop_duplicates(['GeoName' ,'Description'])
p = data.pivot(index = 'GeoName' , columns = 'Description')
CA4_table1 = p.stack()

In [4]:
r = requests.get('http://apps.bea.gov/regional/zip/spi.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
f = z.open('SA4_1929_2017__ALL_AREAS.csv')
SA4_table = pd.read_csv(f,encoding = "ISO-8859-1", low_memory=False)
keep = np.arange(41,94)
keep = keep.tolist()
keep.append(1)
keep.append(6)
data = SA4_table.iloc[:,keep]
data = data[0:2080] #some useless columns
data = data.drop_duplicates(['GeoName' ,'Description'])
p = data.pivot(index = 'GeoName' , columns = 'Description')
SA4_table1 = p.stack()
f = z.open('SA35_1929_2017__ALL_AREAS.csv')
SA35_table = pd.read_csv(f,encoding = "ISO-8859-1", low_memory=False)
keep = np.arange(47,94)
keep = keep.tolist()
keep.append(1)
keep.append(6)
data = SA35_table.iloc[:,keep]
data = data[0:2080] #some useless columns
data = data.drop_duplicates(['GeoName' ,'Description'])
p = data.pivot(index = 'GeoName' , columns = 'Description')
SA35_table1 = p.stack()
f = z.open('SA50_1948_2016__ALL_AREAS.csv')
SA50_table = pd.read_csv(f,encoding = "ISO-8859-1", low_memory=False)
keep = np.arange(28,75)
keep = keep.tolist()
keep.append(1)
keep.append(6)
data = SA50_table.iloc[:,keep]
data = data[0:1199] #some useless columns
data = data.drop_duplicates(['GeoName' ,'Description'])
p = data.pivot(index = 'GeoName' , columns = 'Description')
SA50_table1 = p.stack()

In [5]:
GDP_deflator_annual = Quandl.get("FRED/GDPDEF", authtoken="5QphWABG_zpJsB5dy4yW", collapse="annual")

Construction of GSP: changes time period from 1969, and employment instead of population

In [6]:
r = requests.get('http://apps.bea.gov/regional/zip/gsp/gsp_sic_all.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
f = z.open('gsp_sic_all.csv')
GSP_sic_table = pd.read_csv(f,encoding = "ISO-8859-1", low_memory=False)

In [7]:
r = requests.get('http://apps.bea.gov/regional/zip/gsp/gsp_naics_all.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
f = z.open('gsp_naics_all.csv')
GSP_naics_table = pd.read_csv(f,encoding = "ISO-8859-1", low_memory=False)

In [8]:
Industry_Total_GSP = GSP_sic_table.iloc[::78]
Industry_Total_GSP = Industry_Total_GSP.iloc[1:52]
keep = list(range(8,43))
keep.append(1)
Industry_Total_GSP = Industry_Total_GSP.iloc[:,keep].set_index('GeoName')
Industry_Total_GSP = Industry_Total_GSP.applymap(float)

In [9]:
Industry_Total_GSP_naics = GSP_naics_table.iloc[::90]
Industry_Total_GSP_naics = Industry_Total_GSP_naics.iloc[1:52]
keep = list(range(8,28))
keep.append(1)
Industry_Total_GSP_naics = Industry_Total_GSP_naics.iloc[:,keep].set_index('GeoName')
Industry_Total_GSP_naics = Industry_Total_GSP_naics.applymap(float)
naics_factor = Industry_Total_GSP_naics.values[:,0]/Industry_Total_GSP.values[:,-1]
Industry_Total_GSP_naics = Industry_Total_GSP_naics.drop(columns=['1997'])
Industry_Total_GSP  = (Industry_Total_GSP * np.tile(naics_factor,(35,1)).T)

In [10]:
nominal_Industry_Total_GSP = pd.concat([Industry_Total_GSP, Industry_Total_GSP_naics], axis=1)
nominal_Industry_Total_GSP1 = nominal_Industry_Total_GSP.iloc[:,-48:-1]

In [11]:
real_Industry_Total_GSP = nominal_Industry_Total_GSP / GDP_deflator_annual.iloc[16:70,0].values

In [12]:
names = Industry_Total_GSP.index
names_vector2 = [s + ' state total' for s in names] + [s + ' state total*' for s in names]
CA4_table3 = CA4_table1.loc[names_vector2]

In [13]:
Employment = CA4_table3.iloc[20::22]
Employment = Employment.applymap(float)
Employment_share = Employment /Employment.values.sum(0)

In [14]:
Real_Output_pc = 100* 1000* 1000*real_Industry_Total_GSP.iloc[:,6:-1]/Employment.values

In [15]:
Population = CA4_table3.iloc[17::22]
Population = Population.applymap(float)
Population_share = Population /Population.values.sum(0)

Load federal government finances on the fly:

In [16]:
r = requests.get('https://www.whitehouse.gov/sites/whitehouse.gov/files/omb/budget/fy2018/hist.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
f = z.open('hist02z1.xls')
FED_govt_table_2_1 = pd.read_excel(f,header = [2,3], encoding = "ISO-8859-1") # in millions
FED_govt_table_2_1 = FED_govt_table_2_1.transpose()
FED_govt_table_2_1 = FED_govt_table_2_1.drop(columns = ['TQ'])
f = z.open('hist02z4.xls')
FED_govt_table_2_4 = pd.read_excel(f,header = 2, encoding = "ISO-8859-1")
FED_govt_table_2_4 = FED_govt_table_2_4.drop(columns = ['TQ'])
f = z.open('hist02z5.xls')
FED_govt_table_2_5 = pd.read_excel(f,header = [2,3], encoding = "ISO-8859-1") # in millions
FED_govt_table_2_5 = FED_govt_table_2_5.transpose()
FED_govt_table_2_5 = FED_govt_table_2_5.drop(columns = ['TQ'])

Detailed state government finances -  for now downloaded as it is mostly in access database prior to 1992

In [17]:
#r = requests.get('https://www2.census.gov/pub/outgoing/govs/special60/State_Govt_Fin.zip')
#z = zipfile.ZipFile(io.BytesIO(r.content))
#f = z.open('State_Govt_Finances.mdb')
#accumulator = []
# for now just load the excel files from the .mdb
#for chunk in mdb.read_table(f, '1_Revenues', chunksize=10000):
#    accumulator.append(f(chunk))
# df = mdb.read_table('State_Govt_Finances.mdb', '1_Revenues')

In [18]:
State_Revenues = pd.read_excel('State_Revenues.xlsx')

Load it from 2008 on:

In [20]:
State_Finance_08 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2008/historical-datasets/08state35.txt',header = None)
State_Finance_09 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2009/historical-datasets/09state35.txt',header = None)
State_Finance_10 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2010/historical-datasets/10state35.txt',header = None)
State_Finance_11 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2011/historical-datasets/11state35.txt',header = None)
State_Finance_12 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2012/historical-datasets/12state35.txt',header = None)
State_Finance_13 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2013/historical-datasets/13state35.txt',header = None)
State_Finance_14 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2014/historical-datasets/14state35.txt',header = None)
State_Finance_15 = pd.read_fwf('https://www2.census.gov/programs-surveys/state/datasets/2015/historical-datasets/15state35.txt',header = None)

State_ID =pd.read_excel('https://www2.census.gov/programs-surveys/state/technical-documentation/code-list/government-ids.xls')
Itemcodes =pd.read_excel('https://www2.census.gov/programs-surveys/state/technical-documentation/code-list/itemcodes.xls')
Itemcodes_old =pd.read_excel('https://www2.census.gov/programs-surveys/state/technical-documentation/code-list/itemcodes-old.xls')
#Donwload the correct abbreviations:
State_abbrev =pd.read_excel('http://www.downloadexcelfiles.com/sites/default/files/docs/list_of_u.s._state_abbreviations-1514j.xlsx',header = 1)

Construction of federal nonpersonal taxes and contributions: weights for each state is half personal income, half capital income

In [21]:
total_corporate_income_tax = FED_govt_table_2_1.iloc[1,35:82] 

In [22]:
corp_tax = Employment.copy()

In [23]:
Personal_nominal_income = CA4_table3.iloc[13::22]
Personal_nominal_income = Personal_nominal_income.applymap(float)
Personal_nominal_income_share = Personal_nominal_income /Personal_nominal_income.values.sum(0)

In [24]:
Capital_nominal_income = CA4_table3.iloc[15::22]
Capital_nominal_income = Capital_nominal_income.applymap(float)
Capital_nominal_income_share = Capital_nominal_income /Capital_nominal_income.values.sum(0)

In [25]:
weights_corp_tax = (Capital_nominal_income_share.values + Personal_nominal_income_share.values)/2

In [26]:
corp_tax.iloc[:,:] =  (weights_corp_tax * np.tile(total_corporate_income_tax.T,(51,1)))

In [27]:
#corp_tax = corp_tax  /Population.values 
#corp_tax

In [28]:
#corp_tax = corp_tax  /GDP_deflator_annual.iloc[22:69,0].values * 100

Construction of tobacco/excise tax: weights for each state is half personal income, half pop_share

In [29]:
alcohol_tax = FED_govt_table_2_4.iloc[23,29:76]
estate_gift_tax = FED_govt_table_2_5.iloc[1,29:76]
customs_tax = FED_govt_table_2_5.iloc[2,29:76]
misc_tax = FED_govt_table_2_5.iloc[5,29:76]
highway_tax = FED_govt_table_2_4.iloc[36,29:76]
airway_tax = FED_govt_table_2_4.iloc[37,29:76]
airway_tax = airway_tax.replace(to_replace='..........', value=0)
crude_oil_tax = FED_govt_table_2_4.iloc[25,29:76]
crude_oil_tax = crude_oil_tax.replace(to_replace='..........', value=0)
telephone_tax = FED_govt_table_2_4.iloc[26,29:76]
telephone_tax = telephone_tax.replace(to_replace='..........', value=0)
black_lung_tax = FED_govt_table_2_4.iloc[38,29:76]
black_lung_tax = black_lung_tax.replace(to_replace='..........', value=0)
hazard_tax = FED_govt_table_2_4.iloc[40,29:76]
hazard_tax = hazard_tax.replace(to_replace='..........', value=0)
total_excise_tax = (alcohol_tax + estate_gift_tax + customs_tax + misc_tax + highway_tax + airway_tax + crude_oil_tax +
                   telephone_tax + black_lung_tax + hazard_tax)

In [30]:
total_tobacco_tax = FED_govt_table_2_4.iloc[24,29:76]

In [31]:
excise_tax = Employment.copy()
weights_excise_tax = Personal_nominal_income_share.values

In [32]:
excise_tax.iloc[:,:] = (weights_excise_tax * np.tile(total_excise_tax.T,(51,1)))

In [33]:
#excise_tax = excise_tax  /Employment.values 
#excise_tax = excise_tax  /GDP_deflator_annual.iloc[22:69,0].values * 100

In [34]:
tobacco_tax = Employment.copy()
weights_tobacco_tax = (Personal_nominal_income_share.values + Population_share.values)/2

In [35]:
tobacco_tax.iloc[:,:] = (weights_tobacco_tax * np.tile(total_tobacco_tax.T,(51,1)))

Construction of social security contributions: weights for each state is half personal income, half SS contributions

In [36]:
federal_employment_contr = FED_govt_table_2_4.iloc[11,29:76]
federal_UI_contr = FED_govt_table_2_4.iloc[15,29:76]
federal_other_reti_contr = FED_govt_table_2_4.iloc[19,29:76]
aggr_ss_contr = federal_employment_contr + federal_UI_contr + federal_other_reti_contr

In [37]:
personal_social_security_contributions = CA4_table3.iloc[0::22]
personal_social_security_contributions = personal_social_security_contributions.applymap(float)
personal_social_security_contributions_share = personal_social_security_contributions /personal_social_security_contributions.values.sum(0)

In [38]:
weights_ss_contr = (Personal_nominal_income_share.values + personal_social_security_contributions_share)/2

In [39]:
ss_contr =  Employment.copy()

In [40]:
ss_contr = weights_ss_contr * np.tile(aggr_ss_contr.T,(51,1))

Construction of state unemployment contributions

In [41]:
Total_UI_rev = State_Revenues.iloc[:,-13]
interest_UI_rev = State_Revenues.iloc[:,-11]
UI_tax = Total_UI_rev - interest_UI_rev
UI_tax_df = State_Revenues.copy()
UI_tax_df = UI_tax_df.iloc[:,[2,7,-13]]
UI_tax_df.rename(columns={'Total Unemp Rev': 'UI_tax'}, inplace=True)
UI_tax_df['UI_tax'] = UI_tax
UI_tax_df = UI_tax_df.pivot(index = 'Name' , columns = 'Year4')
UI_tax_df.index.values[9] = 'DC STATE GOVT'
index_name = UI_tax_df.index.copy()
index_name = np.delete(index_name, [1,13,46], None)
UI_tax_df = UI_tax_df.reindex(index_name)
UI_tax_df.index.values[:] = UI_tax_df.index.astype(str).str[0:2]
UI_tax_df = UI_tax_df.reindex(State_abbrev.iloc[3:54,3].values)
# Set DC values to 0 after 2008
UI_tax_df.iloc[8,71] = 0
# 2009
State_Finance_09['State'] = State_Finance_09[0].astype(str).str[0:14]
State_Finance_09['Item'] = State_Finance_09[0].astype(str).str[14:]
State_Finance_09['Dollar Amount'] = State_Finance_09[1].astype(str).str[:-6]
State_Finance_09 = State_Finance_09.drop(columns = [0,1])
State_Finance_09_pivot = State_Finance_09.pivot(index = 'State' , columns = 'Item')
State_Finance_09_pivot.index.values[1:] = State_ID['State']
UI_09_Y01 = np.array(list(map(int, State_Finance_09_pivot['Dollar Amount']['Y01'].values)))
State_Finance_09_pivot['Dollar Amount']['Y04'].values[:] = State_Finance_09_pivot['Dollar Amount']['Y04'].fillna(value=0)
UI_09_Y04 = np.array(list(map(int, State_Finance_09_pivot['Dollar Amount']['Y04'].values)))
UI_09 = UI_09_Y01 + UI_09_Y04
UI_09_aug = np.zeros((51))
UI_09_aug[:8] = UI_09[1:9]
UI_09_aug[9:] = UI_09[9:]
UI_tax_df['UI_tax',2009] = UI_09_aug
# 2010
State_Finance_10['State'] = State_Finance_10[0].astype(str).str[0:14]
State_Finance_10['Item'] = State_Finance_10[0].astype(str).str[14:]
State_Finance_10['Dollar Amount'] = State_Finance_10[1].astype(str).str[:-6]
State_Finance_10 = State_Finance_10.drop(columns = [0,1])
State_Finance_10_pivot = State_Finance_10.pivot(index = 'State' , columns = 'Item')
State_Finance_10_pivot.index.values[1:] = State_ID['State']
UI_10_Y01 = np.array(list(map(int, State_Finance_10_pivot['Dollar Amount']['Y01'].values)))
State_Finance_10_pivot['Dollar Amount']['Y04'].values[:] = State_Finance_10_pivot['Dollar Amount']['Y04'].fillna(value=0)
UI_10_Y04 = np.array(list(map(int, State_Finance_10_pivot['Dollar Amount']['Y04'].values)))
UI_10 = UI_10_Y01 + UI_10_Y04
UI_10_aug = np.zeros((51))
UI_10_aug[:8] = UI_10[1:9]
UI_10_aug[9:] = UI_10[9:]
UI_tax_df['UI_tax',2010] = UI_10_aug
# 2011
State_Finance_11['State'] = State_Finance_11[0].astype(str).str[0:14]
State_Finance_11['Item'] = State_Finance_11[0].astype(str).str[14:]
State_Finance_11['Dollar Amount'] = State_Finance_11[1].astype(str).str[:-6]
State_Finance_11 = State_Finance_11.drop(columns = [0,1])
State_Finance_11_pivot = State_Finance_11.pivot(index = 'State' , columns = 'Item')
State_Finance_11_pivot.index.values[1:] = State_ID['State']
UI_11_Y01 = np.array(list(map(int, State_Finance_11_pivot['Dollar Amount']['Y01'].values)))
State_Finance_11_pivot['Dollar Amount']['Y04'].values[:] = State_Finance_11_pivot['Dollar Amount']['Y04'].fillna(value=0)
UI_11_Y04 = np.array(list(map(int, State_Finance_11_pivot['Dollar Amount']['Y04'].values)))
UI_11 = UI_11_Y01 + UI_11_Y04
UI_11_aug = np.zeros((51))
UI_11_aug[:8] = UI_11[1:9]
UI_11_aug[9:] = UI_11[9:]
UI_tax_df['UI_tax',2011] = UI_11_aug
# 2012
State_Finance_12['State'] = State_Finance_12[0].astype(str).str[0:14]
State_Finance_12['Item'] = State_Finance_12[0].astype(str).str[14:]
State_Finance_12['Dollar Amount'] = State_Finance_12[1].astype(str).str[:-6]
State_Finance_12 = State_Finance_12.drop(columns = [0,1])
State_Finance_12_pivot = State_Finance_12.pivot(index = 'State' , columns = 'Item')
State_Finance_12_pivot.index.values[1:] = State_ID['State']
UI_12_Y01 = np.array(list(map(int, State_Finance_12_pivot['Dollar Amount']['Y01'].values)))
State_Finance_12_pivot['Dollar Amount']['Y04'].values[:] = State_Finance_12_pivot['Dollar Amount']['Y04'].fillna(value=0)
UI_12_Y04 = np.array(list(map(int, State_Finance_12_pivot['Dollar Amount']['Y04'].values)))
UI_12 = UI_12_Y01 + UI_12_Y04
UI_12_aug = np.zeros((51))
UI_12_aug[:8] = UI_12[1:9]
UI_12_aug[9:] = UI_12[9:]
UI_tax_df['UI_tax',2012] = UI_12_aug
# 2013
State_Finance_13['State'] = State_Finance_13[0].astype(str).str[0:14]
State_Finance_13['Item'] = State_Finance_13[0].astype(str).str[14:]
State_Finance_13['Dollar Amount'] = State_Finance_13[1].astype(str).str[:-6]
State_Finance_13 = State_Finance_13.drop(columns = [0,1])
#State_Finance_13_pivot = State_Finance_13.pivot(index = 'State' , columns = 'Item')
State_Finance_13_pivot = State_Finance_13.groupby(['State','Item']).first().unstack()
State_Finance_13_pivot.index.values[1:] = State_ID['State']
UI_13_Y01 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['Y01'].values)))
State_Finance_13_pivot['Dollar Amount']['Y04'].values[:] = State_Finance_13_pivot['Dollar Amount']['Y04'].fillna(value=0)
UI_13_Y04 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['Y04'].values)))
UI_13 = UI_13_Y01 + UI_13_Y04
UI_13_aug = np.zeros((51))
UI_13_aug[:8] = UI_13[1:9]
UI_13_aug[9:] = UI_13[9:]
UI_tax_df['UI_tax',2013] = UI_13_aug
# 2014
State_Finance_14['State'] = State_Finance_14[0].astype(str).str[0:14]
State_Finance_14['Item'] = State_Finance_14[0].astype(str).str[14:]
State_Finance_14['Dollar Amount'] = State_Finance_14[1].astype(str).str[:-6]
State_Finance_14 = State_Finance_14.drop(columns = [0,1])
State_Finance_14['Dollar Amount'].replace('',0, inplace=True)
#State_Finance_14_pivot = State_Finance_14.pivot(index = 'State' , columns = 'Item')
State_Finance_14_pivot = (State_Finance_14.groupby(['State','Item'])
   .first()
   .unstack()
)
State_Finance_14_pivot.index.values[1:] = State_ID['State']
UI_14_Y01 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['Y01'].values)))
State_Finance_14_pivot['Dollar Amount']['Y04'].values[:] = State_Finance_14_pivot['Dollar Amount']['Y04'].replace('', 0, regex=True)
UI_14_Y04 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['Y04'].values)))
UI_14 = UI_14_Y01 + UI_14_Y04
UI_14_aug = np.zeros((51))
UI_14_aug[:8] = UI_14[1:9]
UI_14_aug[9:] = UI_14[9:]
UI_tax_df['UI_tax',2014] = UI_14_aug
# 2015
State_Finance_15['State'] = State_Finance_15[0].astype(str).str[0:14]
State_Finance_15['Item'] = State_Finance_15[0].astype(str).str[14:]
State_Finance_15['Dollar Amount'] = State_Finance_15[1].astype(str).str[:-6]
State_Finance_15 = State_Finance_15.drop(columns = [0,1])
#State_Finance_15_pivot = State_Finance_15.pivot(index = 'State' , columns = 'Item')
State_Finance_15_pivot = (State_Finance_15.groupby(['State','Item'])
   .first()
   .unstack()
)
State_Finance_15_pivot.index.values[1:9] = State_ID['State'][:8]
State_Finance_15_pivot.index.values[9:] = State_ID['State'][8:]
State_Finance_15_pivot.drop(State_Finance_15_pivot.index[9], inplace=True)
UI_15_Y01 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['Y01'].values)))
State_Finance_15_pivot['Dollar Amount']['Y04'].values[:] = State_Finance_15_pivot['Dollar Amount']['Y04'].replace('', 0, regex=True)
UI_15_Y04 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['Y04'].values)))
UI_15 = UI_15_Y01 + UI_15_Y04
UI_15_aug = np.zeros((51))
UI_15_aug[:8] = UI_15[1:9]
UI_15_aug[10:] = UI_15[9:]
UI_tax_df['UI_tax',2015] = UI_15_aug

In [42]:
federal_non_personal_taxes = corp_tax + tobacco_tax.values + ss_contr.values + excise_tax.values + UI_tax_df.iloc[:,32:].values/1000

\section*{Construction of state and local non-personal taxes}

In [43]:
# Downloaded from https://www2.census.gov/pub/outgoing/govs/special60/Govt_Finances.zip

In [44]:
Govt_Revenues = pd.read_excel('Govt_Revenues.xlsx')

In [45]:
State_local_tax = Govt_Revenues.iloc[:,19]
State_local_tax_df = Govt_Revenues.copy()
State_local_tax_df = State_local_tax_df.iloc[:,[2,6,19]]
State_local_tax_df.rename(columns={'Total Taxes': 'State & local tax'}, inplace=True)
State_local_tax_df1 = State_local_tax_df[State_local_tax_df['Name'].str.contains('STATE-LOCAL TOTAL')==True]
State_local_tax_df_pivot = State_local_tax_df1.pivot(index = 'Name' , columns = 'Year4')
index_name = State_local_tax_df_pivot.index.copy()
index_name = np.delete(index_name, [1,13,46], None)
State_local_tax_df_pivot = State_local_tax_df_pivot.reindex(index_name)
State_local_tax_df_pivot.index.values[:] = State_local_tax_df_pivot.index.astype(str).str[0:2]
State_local_tax_df_pivot = State_local_tax_df_pivot.reindex(State_abbrev.iloc[3:54,3].values)

In [46]:
# Correct for the missing data: 2001 & 2003 - total local tax distributed across state as in the previous year
US_total_local_tax = State_local_tax_df[State_local_tax_df['Name'].str.contains('US LOCAL TOTAL')==True]
Local_tax_df = State_local_tax_df[State_local_tax_df['Name'].str.contains('LOCAL TOTAL')==True]
Local_tax_df = Local_tax_df[Local_tax_df['Name'].str.contains('STATE')==False]
US_total_local_tax_2001 = US_total_local_tax.iloc[7,2]
US_total_local_tax_2003 = US_total_local_tax.iloc[5,2]
Local_tax_df_pivot = Local_tax_df.pivot(index = 'Name' , columns = 'Year4')
index_name = Local_tax_df_pivot.index.copy()
index_name = np.delete(index_name, [44], None)
Local_tax_df_pivot = Local_tax_df_pivot.reindex(index_name)
Local_tax_df_pivot.index.values[:] = Local_tax_df_pivot.index.astype(str).str[0:2]
Local_tax_df_pivot = Local_tax_df_pivot.reindex(State_abbrev.iloc[3:54,3].values)
Local_tax_df_pivot['State & local tax'][2001].values[:] = Local_tax_df_pivot['State & local tax'][2000].values/Local_tax_df_pivot['State & local tax'][2000].values.sum() * US_total_local_tax_2001
Local_tax_df_pivot['State & local tax'][2003].values[:] = Local_tax_df_pivot['State & local tax'][2002].values/Local_tax_df_pivot['State & local tax'][2002].values.sum() * US_total_local_tax_2003
State_tax_df = State_local_tax_df[State_local_tax_df['Name'].str.contains('STATE GOVT')==True]
State_tax_df = State_tax_df[State_tax_df['Name'].str.contains('US')==False]
State_tax_df_pivot = State_tax_df.pivot(index = 'Name' , columns = 'Year4')
State_tax_df_pivot.index.values[:] = State_tax_df_pivot.index.astype(str).str[0:2]
State_tax_df_pivot = State_tax_df_pivot.reindex(State_abbrev.iloc[3:54,3].values)
State_local_tax_df_pivot['State & local tax'][2003].values[:] =  Local_tax_df_pivot['State & local tax'][2003].values[:] + State_tax_df_pivot['State & local tax'][2003].values[:]
State_local_tax_df_pivot['State & local tax'][2001].values[:] =  Local_tax_df_pivot['State & local tax'][2001].values[:] + State_tax_df_pivot['State & local tax'][2001].values[:]


In [47]:
# Data from 2009-2015
r_09 =  pd.read_excel('http://www2.census.gov/govs/local/09slsstab1a.xls')
l_09 =  pd.read_excel('http://www2.census.gov/govs/local/09slsstab1b.xls')
State_Local_Finance_09 = pd.concat([r_09, l_09], axis=1)
State_local_tax_df_pivot['State & local tax',2009] =pd.concat([State_Local_Finance_09.iloc[22,6:130:5],State_Local_Finance_09.iloc[22,132::5]] , axis=0).values[:]
r_10 =  pd.read_excel('http://www2.census.gov/govs/local/10slsstab1a.xls')
l_10 =  pd.read_excel('http://www2.census.gov/govs/local/10slsstab1b.xls')
State_Local_Finance_10 = pd.concat([r_10, l_10], axis=1)
State_local_tax_df_pivot['State & local tax',2010] =pd.concat([State_Local_Finance_10.iloc[22,6:130:5],State_Local_Finance_10.iloc[22,132::5]] , axis=0).values[:]
r_11 =  pd.read_excel('http://www2.census.gov/govs/local/11slsstab1a.xls')
l_11 =  pd.read_excel('http://www2.census.gov/govs/local/11slsstab1b.xls')
State_Local_Finance_11 = pd.concat([r_11, l_11], axis=1)
State_local_tax_df_pivot['State & local tax',2011] =pd.concat([State_Local_Finance_11.iloc[22,6:130:5],State_Local_Finance_11.iloc[22,132::5]] , axis=0).values[:]
r_12 =  pd.read_excel('http://www2.census.gov/govs/local/12slsstab1a.xls',header = None)
l_12 =  pd.read_excel('http://www2.census.gov/govs/local/12slsstab1b.xls',header = None)
State_Local_Finance_12 = pd.concat([r_12, l_12], axis=1)
State_local_tax_df_pivot['State & local tax',2012] =pd.concat([State_Local_Finance_12.iloc[22,5:80:3],State_Local_Finance_12.iloc[22,82::3]] , axis=0).values[:]
r_13 =  pd.read_excel('http://www2.census.gov/govs/local/13slsstab1a.xls',header = None)
l_13 =  pd.read_excel('http://www2.census.gov/govs/local/13slsstab1b.xls',header = None)
State_Local_Finance_13 = pd.concat([r_13, l_13], axis=1)
State_local_tax_df_pivot['State & local tax',2013] =pd.concat([State_Local_Finance_13.iloc[24,7:130:5],State_Local_Finance_13.iloc[24,134::5]] , axis=0).values[:]
r_14 =  pd.read_excel('http://www2.census.gov/govs/local/14slsstab1a.xls',header = None)
l_14 =  pd.read_excel('http://www2.census.gov/govs/local/14slsstab1b.xls',header = None)
State_Local_Finance_14 = pd.concat([r_14, l_14], axis=1)
State_local_tax_df_pivot['State & local tax',2014] =pd.concat([State_Local_Finance_14.iloc[24,7:130:5],State_Local_Finance_14.iloc[24,134::5]] , axis=0).values[:]
r_15 =  pd.read_excel('http://www2.census.gov/govs/local/15slsstab1a.xlsx',header = None)
l_15 =  pd.read_excel('http://www2.census.gov/govs/local/15slsstab1b.xlsx',header = None)
State_Local_Finance_15 = pd.concat([r_15, l_15], axis=1)
State_local_tax_df_pivot['State & local tax',2015] =pd.concat([State_Local_Finance_15.iloc[25,7:130:5],State_Local_Finance_15.iloc[25,134::5]] , axis=0).values[:]

Use State Personal Income accounts to contruct personal taxes

In [48]:
names_vector2 = [s for s in names] + [s + '*' for s in names]
SA50_table3 = SA50_table1.loc[names_vector2]

In [49]:
Personal_tax_nontax_payments_state = SA50_table3[11::18]
Personal_tax_nontax_payments_local = SA50_table3[9::18]
State_local_personal_property_taxes = SA50_table3[10::18]

In [50]:
Personal_tax_nontax_payments_local_state = (Personal_tax_nontax_payments_state + Personal_tax_nontax_payments_local.values + 
                                            State_local_personal_property_taxes.values)

In [51]:
state_local_non_personal_taxes = (State_local_tax_df_pivot.iloc[:,-47:] - Personal_tax_nontax_payments_local_state.values)/ 1000

Interest on state and local funds

Only state financing data is used, no local governments. This omission is because I couldnt find a reliable source for this data
(yet). Anyway, the local part seems to be the less important part of these interest earning (comparing it to the original paper).  Simply subtract Y02 before 2008 and not add before 2008.
Update 1 - trust interest is now both local and state added before 2008.

In [52]:
interest_state_insurance_df = State_Revenues.copy()
interest_state_insurance_df = interest_state_insurance_df.iloc[:,[2,7,156,159,164]]
interest_state_insurance_df.iloc[:,2].values[:] = (interest_state_insurance_df.iloc[:,2].values + # Y04
                                                   interest_state_insurance_df.iloc[:,3].values + 
                                                   interest_state_insurance_df.iloc[:,4].values)- interest_UI_rev
interest_state_insurance_df = interest_state_insurance_df.drop(columns=['Worker Comp-Oth Ctrib (Y11)'])
interest_state_insurance_df = interest_state_insurance_df.drop(columns=['Oth In Trust-Oth Ctrib (Y51)'])
interest_state_insurance_df = interest_state_insurance_df.pivot(index = 'Name' , columns = 'Year4')
interest_state_insurance_df.index.values[9] = 'DC STATE GOVT'
index_name = interest_state_insurance_df.index.copy()
index_name = np.delete(index_name, [1,13,46], None)
interest_state_insurance_df = interest_state_insurance_df.reindex(index_name)
interest_state_insurance_df.index.values[:] = interest_state_insurance_df.index.astype(str).str[0:2]
interest_state_insurance_df = interest_state_insurance_df.reindex(State_abbrev.iloc[3:54,3].values)
interest_state_insurance_df.iloc[8,71] = 0

In [53]:
interest_state_insurance_govt_df = Govt_Revenues.copy()
interest_state_insurance_govt_df = interest_state_insurance_govt_df.iloc[:,[2,6,92,94,95,96]]
interest_state_insurance_govt_df.iloc[:,2].values[:] = (interest_state_insurance_govt_df.iloc[:,2].values +
                                                       interest_state_insurance_govt_df.iloc[:,3].values - 
                                                       interest_state_insurance_govt_df.iloc[:,4].values + 
                                                       interest_state_insurance_govt_df.iloc[:,5].values)
interest_state_insurance_govt_df = interest_state_insurance_govt_df.iloc[:,:3]
interest_state_insurance_govt_df1 = interest_state_insurance_govt_df[interest_state_insurance_govt_df[
    'Name'].str.contains('STATE-LOCAL TOTAL')==True]
interest_state_insurance_govt_df_pivot = interest_state_insurance_govt_df1.pivot(index = 'Name' , columns = 'Year4')
index_name = interest_state_insurance_govt_df_pivot.index.copy()
index_name = np.delete(index_name, [1,13,46], None)
interest_state_insurance_govt_df_pivot = interest_state_insurance_govt_df_pivot.reindex(index_name)
interest_state_insurance_govt_df_pivot.index.values[:] = interest_state_insurance_govt_df_pivot.index.astype(str).str[0:2]
interest_state_insurance_govt_df_pivot = interest_state_insurance_govt_df_pivot.reindex(State_abbrev.iloc[3:54,3].values)

Correct for 2001 and 2003

In [54]:
US_total_interest_state_insurance_govt= interest_state_insurance_govt_df[interest_state_insurance_govt_df['Name'].str.contains('US STATE-LOCAL TOTAL')==True]
US_total_interest_state_insurance_govt_2001 = US_total_interest_state_insurance_govt.iloc[7,2]
US_total_interest_state_insurance_govt_2003 = US_total_interest_state_insurance_govt.iloc[5,2]
interest_state_insurance_govt_df_pivot['Emp Ret-Int Rev (X08)'][2003].values[:] =  (interest_state_insurance_govt_df_pivot['Emp Ret-Int Rev (X08)'][2002].values[:]/
                                                                interest_state_insurance_govt_df_pivot['Emp Ret-Int Rev (X08)'][2002].values.sum() *
                                                                US_total_interest_state_insurance_govt_2003)
interest_state_insurance_govt_df_pivot['Emp Ret-Int Rev (X08)'][2001].values[:] =  (interest_state_insurance_govt_df_pivot['Emp Ret-Int Rev (X08)'][2000].values[:]/
                                                                interest_state_insurance_govt_df_pivot['Emp Ret-Int Rev (X08)'][2000].values.sum() * 
                                                               US_total_interest_state_insurance_govt_2001)

In [55]:
interest_state_insurance_df = interest_state_insurance_govt_df_pivot.iloc[:,-40:] - interest_state_insurance_df.iloc[:,-40:].values

In [56]:
# 2009 
interest_state_insurance_09_X08 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['X08'].values)))
#State_Finance_09_pivot['Dollar Amount']['Y02'].values[:] = State_Finance_09_pivot['Dollar Amount']['Y02'].fillna(value=0)
#interest_state_insurance_09_Y02 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['Y02'].values)))
State_Finance_09_pivot['Dollar Amount']['Y12'].values[:] = State_Finance_09_pivot['Dollar Amount']['Y12'].fillna(value=0)
interest_state_insurance_09_Y12 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['Y12'].values)))
State_Finance_09_pivot['Dollar Amount']['Y52'].values[:] = State_Finance_09_pivot['Dollar Amount']['Y52'].fillna(value=0)
interest_state_insurance_09_Y52 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['Y52'].values)))
interest_state_insurance_09 = (interest_state_insurance_09_X08  + interest_state_insurance_09_Y12
         + interest_state_insurance_09_Y52)
interest_state_insurance_09_aug = np.zeros((51))
interest_state_insurance_09_aug[:8] = interest_state_insurance_09[1:9]
interest_state_insurance_09_aug[9:] = interest_state_insurance_09[9:]
interest_state_insurance_df['Tot Ins Trust Inv Rev',2009] = interest_state_insurance_09_aug
# 2010 
interest_state_insurance_10_X08 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['X08'].values)))
#State_Finance_10_pivot['Dollar Amount']['Y02'].values[:] = State_Finance_10_pivot['Dollar Amount']['Y02'].fillna(value=0)
#interest_state_insurance_10_Y02 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['Y02'].values)))
State_Finance_10_pivot['Dollar Amount']['Y12'].values[:] = State_Finance_10_pivot['Dollar Amount']['Y12'].fillna(value=0)
interest_state_insurance_10_Y12 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['Y12'].values)))
State_Finance_10_pivot['Dollar Amount']['Y52'].values[:] = State_Finance_10_pivot['Dollar Amount']['Y52'].fillna(value=0)
interest_state_insurance_10_Y52 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['Y52'].values)))
interest_state_insurance_10 = (interest_state_insurance_10_X08  + interest_state_insurance_10_Y12
         + interest_state_insurance_10_Y52)
interest_state_insurance_10_aug = np.zeros((51))
interest_state_insurance_10_aug[:8] = interest_state_insurance_10[1:9]
interest_state_insurance_10_aug[9:] = interest_state_insurance_10[9:]
interest_state_insurance_df['Tot Ins Trust Inv Rev',2010] = interest_state_insurance_10_aug
# 2011
interest_state_insurance_11_X08 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['X08'].values)))
#State_Finance_11_pivot['Dollar Amount']['Y02'].values[:] = State_Finance_11_pivot['Dollar Amount']['Y02'].fillna(value=0)
#interest_state_insurance_11_Y02 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['Y02'].values)))
State_Finance_11_pivot['Dollar Amount']['Y12'].values[:] = State_Finance_11_pivot['Dollar Amount']['Y12'].fillna(value=0)
interest_state_insurance_11_Y12 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['Y12'].values)))
State_Finance_11_pivot['Dollar Amount']['Y52'].values[:] = State_Finance_11_pivot['Dollar Amount']['Y52'].fillna(value=0)
interest_state_insurance_11_Y52 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['Y52'].values)))
interest_state_insurance_11 = (interest_state_insurance_11_X08  + interest_state_insurance_11_Y12
         + interest_state_insurance_11_Y52)
interest_state_insurance_11_aug = np.zeros((51))
interest_state_insurance_11_aug[:8] = interest_state_insurance_11[1:9]
interest_state_insurance_11_aug[9:] = interest_state_insurance_11[9:]
interest_state_insurance_df['Tot Ins Trust Inv Rev',2011] = interest_state_insurance_11_aug
# 2012
interest_state_insurance_12_X08 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['X08'].values)))
#State_Finance_12_pivot['Dollar Amount']['Y02'].values[:] = State_Finance_12_pivot['Dollar Amount']['Y02'].fillna(value=0)
#interest_state_insurance_12_Y02 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['Y02'].values)))
State_Finance_12_pivot['Dollar Amount']['Y12'].values[:] = State_Finance_12_pivot['Dollar Amount']['Y12'].fillna(value=0)
interest_state_insurance_12_Y12 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['Y12'].values)))
State_Finance_12_pivot['Dollar Amount']['Y52'].values[:] = State_Finance_12_pivot['Dollar Amount']['Y52'].fillna(value=0)
interest_state_insurance_12_Y52 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['Y52'].values)))
interest_state_insurance_12 = (interest_state_insurance_12_X08  + interest_state_insurance_12_Y12
         + interest_state_insurance_12_Y52)
interest_state_insurance_12_aug = np.zeros((51))
interest_state_insurance_12_aug[:8] = interest_state_insurance_12[1:9]
interest_state_insurance_12_aug[9:] = interest_state_insurance_12[9:]
interest_state_insurance_df['Tot Ins Trust Inv Rev',2012] = interest_state_insurance_12_aug
# 2013
interest_state_insurance_13_X08 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['X08'].values)))
#State_Finance_13_pivot['Dollar Amount']['Y02'].values[:] = State_Finance_13_pivot['Dollar Amount']['Y02'].fillna(value=0)
#interest_state_insurance_13_Y02 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['Y02'].values)))
State_Finance_13_pivot['Dollar Amount']['Y12'].values[:] = State_Finance_13_pivot['Dollar Amount']['Y12'].fillna(value=0)
interest_state_insurance_13_Y12 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['Y12'].values)))
State_Finance_13_pivot['Dollar Amount']['Y52'].values[:] = State_Finance_13_pivot['Dollar Amount']['Y52'].fillna(value=0)
interest_state_insurance_13_Y52 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['Y52'].values)))
interest_state_insurance_13 = (interest_state_insurance_13_X08  + interest_state_insurance_13_Y12
         + interest_state_insurance_13_Y52)
interest_state_insurance_13_aug = np.zeros((51))
interest_state_insurance_13_aug[:8] = interest_state_insurance_13[1:9]
interest_state_insurance_13_aug[9:] = interest_state_insurance_13[9:]
interest_state_insurance_df['Tot Ins Trust Inv Rev',2013] = interest_state_insurance_13_aug
# 2014
interest_state_insurance_14_X08 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['X08'].values)))
#State_Finance_14_pivot['Dollar Amount']['Y02'].values[:] = State_Finance_14_pivot['Dollar Amount']['Y02'].replace('', 0, regex=True)
#interest_state_insurance_14_Y02 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['Y02'].values)))
State_Finance_14_pivot['Dollar Amount']['Y12'].values[:] = State_Finance_14_pivot['Dollar Amount']['Y12'].fillna(value=0).replace('', 0, regex=True)
interest_state_insurance_14_Y12 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['Y12'].values)))
State_Finance_14_pivot['Dollar Amount']['Y52'].values[:] = State_Finance_14_pivot['Dollar Amount']['Y52'].fillna(value=0).replace('', 0, regex=True)
interest_state_insurance_14_Y52 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['Y52'].values)))
interest_state_insurance_14 = (interest_state_insurance_14_X08  + interest_state_insurance_14_Y12
         + interest_state_insurance_14_Y52)
interest_state_insurance_14_aug = np.zeros((51))
interest_state_insurance_14_aug[:8] = interest_state_insurance_14[1:9]
interest_state_insurance_14_aug[9:] = interest_state_insurance_14[9:]
interest_state_insurance_df['Tot Ins Trust Inv Rev',2014] = interest_state_insurance_14_aug
# 2015
interest_state_insurance_15_X08 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['X08'].values)))
#State_Finance_15_pivot['Dollar Amount']['Y02'].values[:] = State_Finance_15_pivot['Dollar Amount']['Y02'].replace('', 0, regex=True)
#interest_state_insurance_15_Y02 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['Y02'].values)))
State_Finance_15_pivot['Dollar Amount']['Y12'].values[:] = State_Finance_15_pivot['Dollar Amount']['Y12'].fillna(value=0).replace('', 0, regex=True)
interest_state_insurance_15_Y12 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['Y12'].values)))
State_Finance_15_pivot['Dollar Amount']['Y52'].values[:] = State_Finance_15_pivot['Dollar Amount']['Y52'].fillna(value=0).replace('', 0, regex=True)
interest_state_insurance_15_Y52 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['Y52'].values)))
interest_state_insurance_15 = (interest_state_insurance_15_X08  + interest_state_insurance_15_Y12
         + interest_state_insurance_15_Y52)
interest_state_insurance_15_aug = np.zeros((51))
interest_state_insurance_15_aug[:8] = interest_state_insurance_15[1:9]
interest_state_insurance_15_aug[10:] = interest_state_insurance_15[9:]
interest_state_insurance_df['Tot Ins Trust Inv Rev',2015] = interest_state_insurance_15_aug

Miscellaneous interest earnings =  U20 + U21 + U40 + U41. Again, no local, only state level data - yet

In [57]:
misc_interest = (State_Revenues['Interest Revenue (U20)'].values + State_Revenues['Dividends on Investments (U21)'].values + 
                State_Revenues['Rents (U40)'].values + State_Revenues['Royalties (U41)'].values)
misc_interest_df = State_Revenues.copy()
misc_interest_df = misc_interest_df.iloc[:,[2,7,124]]
misc_interest_df.iloc[:,2].values[:] = misc_interest
misc_interest_df.rename(columns={'Interest Revenue (U20)': 'Misc. interest earnings'}, inplace=True)
misc_interest_df = misc_interest_df.pivot(index = 'Name' , columns = 'Year4')
misc_interest_df.index.values[9] = 'DC STATE GOVT'
index_name = misc_interest_df.index.copy()
index_name = np.delete(index_name, [1,13,46], None)
misc_interest_df = misc_interest_df.reindex(index_name)
misc_interest_df.index.values[:] =misc_interest_df.index.astype(str).str[0:2]
misc_interest_df = misc_interest_df.reindex(State_abbrev.iloc[3:54,3].values)
misc_interest_df.iloc[8,71] = 0
# 2009 
misc_interest_09_U20 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['U20'].values)))
State_Finance_09_pivot['Dollar Amount']['U21'].values[:] = State_Finance_09_pivot['Dollar Amount']['U21'].fillna(value=0)
misc_interest_09_U21 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['U21'].values)))
State_Finance_09_pivot['Dollar Amount']['U40'].values[:] = State_Finance_09_pivot['Dollar Amount']['U40'].fillna(value=0)
misc_interest_09_U40 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['U40'].values)))
State_Finance_09_pivot['Dollar Amount']['U41'].values[:] = State_Finance_09_pivot['Dollar Amount']['U41'].fillna(value=0)
misc_interest_09_U41 = np.array(list(map(float, State_Finance_09_pivot['Dollar Amount']['U41'].values)))
misc_interest_09 = (misc_interest_09_U20 + misc_interest_09_U21 + misc_interest_09_U40
         + misc_interest_09_U41)
misc_interest_09_aug = np.zeros((51))
misc_interest_09_aug[:8] = misc_interest_09[1:9]
misc_interest_09_aug[9:] = misc_interest_09[9:]
misc_interest_df['Misc. interest earnings',2009] = misc_interest_09_aug
# 2010 
misc_interest_10_U20 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['U20'].values)))
State_Finance_10_pivot['Dollar Amount']['U21'].values[:] = State_Finance_10_pivot['Dollar Amount']['U21'].fillna(value=0)
misc_interest_10_U21 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['U21'].values)))
State_Finance_10_pivot['Dollar Amount']['U40'].values[:] = State_Finance_10_pivot['Dollar Amount']['U40'].fillna(value=0)
misc_interest_10_U40 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['U40'].values)))
State_Finance_10_pivot['Dollar Amount']['U41'].values[:] = State_Finance_10_pivot['Dollar Amount']['U41'].fillna(value=0)
misc_interest_10_U41 = np.array(list(map(float, State_Finance_10_pivot['Dollar Amount']['U41'].values)))
misc_interest_10 = (misc_interest_10_U20 + misc_interest_10_U21 + misc_interest_10_U40
         + misc_interest_10_U41)
misc_interest_10_aug = np.zeros((51))
misc_interest_10_aug[:8] = misc_interest_10[1:9]
misc_interest_10_aug[9:] = misc_interest_10[9:]
misc_interest_df['Misc. interest earnings',2010] = misc_interest_10_aug
# 2011
misc_interest_11_U20 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['U20'].values)))
State_Finance_11_pivot['Dollar Amount']['U21'].values[:] = State_Finance_11_pivot['Dollar Amount']['U21'].fillna(value=0)
misc_interest_11_U21 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['U21'].values)))
State_Finance_11_pivot['Dollar Amount']['U40'].values[:] = State_Finance_11_pivot['Dollar Amount']['U40'].fillna(value=0)
misc_interest_11_U40 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['U40'].values)))
State_Finance_11_pivot['Dollar Amount']['U41'].values[:] = State_Finance_11_pivot['Dollar Amount']['U41'].fillna(value=0)
misc_interest_11_U41 = np.array(list(map(float, State_Finance_11_pivot['Dollar Amount']['U41'].values)))
misc_interest_11 = (misc_interest_11_U20 + misc_interest_11_U21 + misc_interest_11_U40
         + misc_interest_11_U41)
misc_interest_11_aug = np.zeros((51))
misc_interest_11_aug[:8] = misc_interest_11[1:9]
misc_interest_11_aug[9:] = misc_interest_11[9:]
misc_interest_df['Misc. interest earnings',2011] = misc_interest_11_aug
# 2012
misc_interest_12_U20 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['U20'].values)))
State_Finance_12_pivot['Dollar Amount']['U21'].values[:] = State_Finance_12_pivot['Dollar Amount']['U21'].fillna(value=0)
misc_interest_12_U21 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['U21'].values)))
State_Finance_12_pivot['Dollar Amount']['U40'].values[:] = State_Finance_12_pivot['Dollar Amount']['U40'].fillna(value=0)
misc_interest_12_U40 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['U40'].values)))
State_Finance_12_pivot['Dollar Amount']['U41'].values[:] = State_Finance_12_pivot['Dollar Amount']['U41'].fillna(value=0)
misc_interest_12_U41 = np.array(list(map(float, State_Finance_12_pivot['Dollar Amount']['U41'].values)))
misc_interest_12 = (misc_interest_12_U20 + misc_interest_12_U21 + misc_interest_12_U40
         + misc_interest_12_U41)
misc_interest_12_aug = np.zeros((51))
misc_interest_12_aug[:8] = misc_interest_12[1:9]
misc_interest_12_aug[9:] = misc_interest_12[9:]
misc_interest_df['Misc. interest earnings',2012] = misc_interest_12_aug
# 2013
misc_interest_13_U20 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['U20'].values)))
State_Finance_13_pivot['Dollar Amount']['U21'].values[:] = State_Finance_13_pivot['Dollar Amount']['U21'].fillna(value=0)
misc_interest_13_U21 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['U21'].values)))
State_Finance_13_pivot['Dollar Amount']['U40'].values[:] = State_Finance_13_pivot['Dollar Amount']['U40'].fillna(value=0)
misc_interest_13_U40 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['U40'].values)))
State_Finance_13_pivot['Dollar Amount']['U41'].values[:] = State_Finance_13_pivot['Dollar Amount']['U41'].fillna(value=0)
misc_interest_13_U41 = np.array(list(map(float, State_Finance_13_pivot['Dollar Amount']['U41'].values)))
misc_interest_13 = (misc_interest_13_U20 + misc_interest_13_U21 + misc_interest_13_U40
         + misc_interest_13_U41)
misc_interest_13_aug = np.zeros((51))
misc_interest_13_aug[:8] = misc_interest_13[1:9]
misc_interest_13_aug[9:] = misc_interest_13[9:]
misc_interest_df['Misc. interest earnings',2013] = misc_interest_13_aug
# 2014
misc_interest_14_U20 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['U20'].values)))
State_Finance_14_pivot['Dollar Amount']['U21'].values[:] = State_Finance_14_pivot['Dollar Amount']['U21'].fillna(value=0).replace('', 0, regex=True)
misc_interest_14_U21 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['U21'].values)))
State_Finance_14_pivot['Dollar Amount']['U40'].values[:] = State_Finance_14_pivot['Dollar Amount']['U40'].fillna(value=0).replace('', 0, regex=True)
misc_interest_14_U40 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['U40'].values)))
State_Finance_14_pivot['Dollar Amount']['U41'].values[:] = State_Finance_14_pivot['Dollar Amount']['U41'].fillna(value=0).replace('', 0, regex=True)
misc_interest_14_U41 = np.array(list(map(float, State_Finance_14_pivot['Dollar Amount']['U41'].values)))
misc_interest_14 = (misc_interest_14_U20 + misc_interest_14_U21 + misc_interest_14_U40
         + misc_interest_14_U41)
misc_interest_14_aug = np.zeros((51))
misc_interest_14_aug[:8] = misc_interest_14[1:9]
misc_interest_14_aug[9:] = misc_interest_14[9:]
misc_interest_df['Misc. interest earnings',2014] = misc_interest_14_aug
# 2015
misc_interest_15_U20 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['U20'].values)))
State_Finance_15_pivot['Dollar Amount']['U21'].values[:] = State_Finance_15_pivot['Dollar Amount']['U21'].fillna(value=0).replace('', 0, regex=True)
misc_interest_15_U21 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['U21'].values)))
State_Finance_15_pivot['Dollar Amount']['U40'].values[:] = State_Finance_15_pivot['Dollar Amount']['U40'].fillna(value=0).replace('', 0, regex=True)
misc_interest_15_U40 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['U40'].values)))
State_Finance_15_pivot['Dollar Amount']['U41'].values[:] = State_Finance_15_pivot['Dollar Amount']['U41'].fillna(value=0).replace('', 0, regex=True)
misc_interest_15_U41 = np.array(list(map(float, State_Finance_15_pivot['Dollar Amount']['U41'].values)))
misc_interest_15 = (misc_interest_15_U20 + misc_interest_15_U21 + misc_interest_15_U40
         + misc_interest_15_U41)
misc_interest_15_aug = np.zeros((51))
misc_interest_15_aug[:8] = misc_interest_15[1:9]
misc_interest_15_aug[10:] = misc_interest_15[9:]
misc_interest_df['Misc. interest earnings',2015] = misc_interest_15_aug

In [58]:
interest_state = misc_interest_df.iloc[:,-47:] + interest_state_insurance_df.values

Transfer payments

In [59]:
names_vector2 = [s for s in names] + [s + '*' for s in names]
SA35_table3 = SA35_table1.loc[names_vector2]

In [60]:
payments_indiv = SA35_table3.iloc[37::40,:].applymap(float)

In [61]:
federal_transfers = SA35_table3.iloc[32::40,:].applymap(float)

In [62]:
state_local_transfers = SA35_table3.iloc[31::40,:].applymap(float)

In [63]:
transfers = (payments_indiv + federal_transfers.values + state_local_transfers.values) / 1000

In [64]:
non_federal_state_income = (Personal_nominal_income/1000 + federal_non_personal_taxes.values + 
                            state_local_non_personal_taxes.values+interest_state.values/ 1000 -transfers.values)

Federal grants

In [65]:
fed_grants = Govt_Revenues.loc[:,'Total Fed IG Revenue']
fed_grants_df = Govt_Revenues.copy()
fed_grants_df = fed_grants_df.iloc[:,[2,6,47]]
fed_grants_df1 = fed_grants_df[fed_grants_df['Name'].str.contains('STATE-LOCAL TOTAL')==True]
fed_grants_df_pivot = fed_grants_df1.pivot(index = 'Name' , columns = 'Year4')
index_name = fed_grants_df_pivot.index.copy()
index_name = np.delete(index_name, [1,13,46], None)
fed_grants_df_pivot = fed_grants_df_pivot.reindex(index_name)
fed_grants_df_pivot.index.values[:] = fed_grants_df_pivot.index.astype(str).str[0:2]
fed_grants_df_pivot = fed_grants_df_pivot.reindex(State_abbrev.iloc[3:54,3].values)

Again, correct for the missing data in 2001 and 2003 the same way as before

In [66]:
US_total_fed_grants= fed_grants_df[fed_grants_df['Name'].str.contains('US STATE GOVTS')==True]
US_total_fed_grants_2001 = US_total_fed_grants.iloc[7,2]
US_total_fed_grants_2003 = US_total_fed_grants.iloc[5,2]
fed_grants_df_pivot['Total Fed IG Revenue'][2003].values[:] =  (fed_grants_df_pivot['Total Fed IG Revenue'][2002].values[:]/
                                                                fed_grants_df_pivot['Total Fed IG Revenue'][2002].values.sum() * US_total_fed_grants_2003)
fed_grants_df_pivot['Total Fed IG Revenue'][2001].values[:] =  (fed_grants_df_pivot['Total Fed IG Revenue'][2000].values[:]/
                                                                fed_grants_df_pivot['Total Fed IG Revenue'][2000].values.sum() * US_total_fed_grants_2001)

In [67]:
# Data from 2009-2015

fed_grants_df_pivot['Total Fed IG Revenue',2009] =pd.concat([State_Local_Finance_09.iloc[17,8:132:5],
                                                               State_Local_Finance_09.iloc[17,134::5]] , axis=0).values[:]
fed_grants_df_pivot['Total Fed IG Revenue',2010] =pd.concat([State_Local_Finance_10.iloc[17,8:132:5],
                                                               State_Local_Finance_10.iloc[17,134::5]] , axis=0).values[:]
fed_grants_df_pivot['Total Fed IG Revenue',2011] =pd.concat([State_Local_Finance_11.iloc[17,8:132:5],
                                                               State_Local_Finance_11.iloc[17,134::5]] , axis=0).values[:]
fed_grants_df_pivot['Total Fed IG Revenue',2012] =pd.concat([State_Local_Finance_12.iloc[17,6:81:3],
                                                             State_Local_Finance_12.iloc[17,83::3]] , axis=0).values[:]
fed_grants_df_pivot['Total Fed IG Revenue',2013] =pd.concat([State_Local_Finance_13.iloc[19,9:132:5],
                                                             State_Local_Finance_13.iloc[19,136::5]] , axis=0).values[:]
fed_grants_df_pivot['Total Fed IG Revenue',2014] =pd.concat([State_Local_Finance_14.iloc[19,9:132:5],
                                                             State_Local_Finance_14.iloc[19,136::5]] , axis=0).values[:]
fed_grants_df_pivot['Total Fed IG Revenue',2015] =pd.concat([State_Local_Finance_15.iloc[20,9:132:5],
                                                             State_Local_Finance_15.iloc[20,136::5]] , axis=0).values[:]


In [68]:
fed_grants_df_pivot = fed_grants_df_pivot.iloc[:,-47:] 

Federal transfers & personal taxes

In [69]:
Social_security_benefits = SA35_table3.iloc[19::40,:].applymap(float)
railroad = SA35_table3.iloc[6::40,:].applymap(float)
worker_comp = SA35_table3.iloc[10::40,:].applymap(float)
medical_benefits = SA35_table3.iloc[28::40,:].applymap(float)
ssi_benefits = SA35_table3.iloc[22::40,:].applymap(float)
snap_benefits = SA35_table3.iloc[21::40,:]
snap_benefits = snap_benefits.replace(to_replace='(L)', value=0) #equivalent to snap_benefits.iloc[21,0] = 0 
snap_benefits = snap_benefits.applymap(float)
other_income_benefits  =  SA35_table3.iloc[17::40,:].applymap(float) # this one is completely different and larger than in ASY
UI_benefits = SA35_table3.iloc[34::40,:].applymap(float)
veteran_benefits = SA35_table3.iloc[35::40,:].applymap(float)
education_benefits = SA35_table3.iloc[26::40,:].applymap(float) # almost double the amount than in ASY
medicaid = SA35_table3.iloc[2::40,:].replace(to_replace='(NA)', value=0).applymap(float) # similar to ASY, but taken from BEA instead
fed_transfers_indiv = (Social_security_benefits.values + railroad.values + worker_comp.values + medical_benefits.values + 
                       ssi_benefits.values + snap_benefits.values + other_income_benefits.values + 
                       UI_benefits.values + veteran_benefits.values + education_benefits.values + 
                      federal_transfers - medicaid.values) / 1000

In [70]:
federal_personal_tax = SA50_table3[7::18]

Disposable state income

In [71]:
disposable_state_income = (non_federal_state_income + fed_grants_df_pivot.values /1000 + fed_transfers_indiv.values - 
                          federal_non_personal_taxes.values - federal_personal_tax.values/1000)

Government expenditure

In [72]:
# Downloaded from https://www2.census.gov/pub/outgoing/govs/special60/Govt_Finances.zip
Govt_expenditures = pd.read_excel('Govt_Expenditure.xlsx')

In [73]:
expenditure_df = Govt_expenditures.copy()
expenditure_df = expenditure_df.iloc[:,[2,6,21]]

#interest_state_insurance_df.iloc[8,71] = 0
expenditure_df1 = expenditure_df[expenditure_df['Name'].str.contains('STATE-LOCAL TOTAL')==True]
expenditure_df_pivot = expenditure_df1.pivot(index = 'Name' , columns = 'Year4')
index_name = expenditure_df_pivot.index.copy()
index_name = np.delete(index_name, [1,13,46], None)
expenditure_df_pivot = expenditure_df_pivot.reindex(index_name)
expenditure_df_pivot.index.values[:] = expenditure_df_pivot.index.astype(str).str[0:2]
expenditure_df_pivot = expenditure_df_pivot.reindex(State_abbrev.iloc[3:54,3].values)

In [74]:
US_total_expenditure= expenditure_df[expenditure_df['Name'].str.contains('US STATE-LOCAL TOTAL')==True]
US_total_expenditure_2001 = US_total_expenditure.iloc[7,2]
US_total_expenditure_2003 = US_total_expenditure.iloc[5,2]
expenditure_df_pivot['Direct General Expend'][2003].values[:] =  (expenditure_df_pivot['Direct General Expend'][2002].values[:]/
                                                                expenditure_df_pivot['Direct General Expend'][2002].values.sum() *
                                                                US_total_expenditure_2003)
expenditure_df_pivot['Direct General Expend'][2001].values[:] =  (expenditure_df_pivot['Direct General Expend'][2000].values[:]/
                                                                expenditure_df_pivot['Direct General Expend'][2000].values.sum() * 
                                                               US_total_expenditure_2001)

In [75]:
# Data from 2009-2015

expenditure_df_pivot['Direct General Expend',2009] =pd.concat([State_Local_Finance_09.iloc[89,6:130:5],
                                                               State_Local_Finance_09.iloc[89,132::5]] , axis=0).values[:]
expenditure_df_pivot['Direct General Expend',2010] =pd.concat([State_Local_Finance_10.iloc[89,6:130:5],
                                                               State_Local_Finance_10.iloc[89,132::5]] , axis=0).values[:]
expenditure_df_pivot['Direct General Expend',2011] =pd.concat([State_Local_Finance_11.iloc[89,6:130:5],
                                                               State_Local_Finance_11.iloc[89,132::5]] , axis=0).values[:]
expenditure_df_pivot['Direct General Expend',2012] =pd.concat([State_Local_Finance_12.iloc[89,5:80:3],
                                                             State_Local_Finance_12.iloc[89,82::3]] , axis=0).values[:]
expenditure_df_pivot['Direct General Expend',2013] =pd.concat([State_Local_Finance_13.iloc[91,7:130:5],
                                                             State_Local_Finance_13.iloc[91,134::5]] , axis=0).values[:]
expenditure_df_pivot['Direct General Expend',2014] =pd.concat([State_Local_Finance_14.iloc[91,7:130:5],
                                                             State_Local_Finance_14.iloc[91,134::5]] , axis=0).values[:]
expenditure_df_pivot['Direct General Expend',2015] =pd.concat([State_Local_Finance_15.iloc[92,7:130:5],
                                                             State_Local_Finance_15.iloc[92,134::5]] , axis=0).values[:]


In [76]:
expenditure_df_pivot = expenditure_df_pivot.iloc[:,-47:] 

In [77]:
state_local_transfers = transfers - fed_transfers_indiv.values

In [78]:
state_gov_cons = expenditure_df_pivot / 1000 - state_local_transfers.values

State shares of personal consumption - here I differ significantly from the original paper, as there have been significant improvements in the state/federal level consumption data. Namely prior to 1997, I am going to use their weights to construct personal consumption, however, after 1997, I will use the values published by the BEA. Prior to 1997 I use the updated series for US total personal consumption.

In [79]:
US_consumption_exp = Quandl.get("FRED/PCEC", authtoken="5QphWABG_zpJsB5dy4yW", collapse="annual", end_date="2017-06-30")

In [80]:
r = requests.get('http://apps.bea.gov/regional/zip/PCEbyState.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
f = z.open('PCE_ALL_AREAS.csv')
PCE_table = pd.read_csv(f,encoding = "ISO-8859-1", low_memory=False)
cons_weights = pd.read_excel('nch16.xls',header = 1)

In [81]:
keep = np.arange(8,28)
keep = keep.tolist()
keep.append(1)
keep.append(7)
data = PCE_table.iloc[:,keep]
data = data[0:1248] #some useless columns
data = data.drop_duplicates(['GeoName' ,'Description'])
p = data.pivot(index = 'GeoName' , columns = 'Description')
PCE_table1 = p.stack()
PCE_table2 = PCE_table1.iloc[23::24,:].reset_index(level=1, drop=True)
PCE_table2.index.names = ['state']
PCE_table2 = PCE_table2.drop(['United States'])
p = cons_weights.pivot(index = 'state' , columns = 'Region')
cons_weights1 = p.stack()
cons_weights1 = cons_weights1.reset_index(level=1, drop=True)
cons_weights1  = cons_weights1.iloc[:,:-2]
cons_weights1.index = PCE_table2.index
PCE_table3 = pd.concat([cons_weights1, PCE_table2], axis=1)
cons_weights2 = PCE_table3 / PCE_table3.values.sum(0)
PCE_table4 = cons_weights2 * np.tile(US_consumption_exp.values[16:-1].T,(51,1)) * 1000
PCE_table5 = PCE_table4.iloc[:,-48:-1]

In [82]:
state_consumption = PCE_table5 + state_gov_cons.values

Load IRS migration rates = use the database provided by Janine Billadello

In [83]:
mig_table_inflow = state_consumption.copy()
mig_table_inflow.iloc[:,:] = 0
mig_table_outflow = mig_table_inflow.copy()
mig_table_nonmig = mig_table_inflow.copy()
state_index_ver = state_consumption.index.str.upper()

In [84]:
r = requests.get('http://faculty.baruch.cuny.edu/geoportal/data/irs_migration/irsmig_state_database.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
f = z.open(z.filelist[3])

In [85]:
State_name_index_abbr = State_abbrev.iloc[3:54,3].values
#State_name_index_abbr = [s + ' Total Migration-US and Foreign' for s in State_name_index_abbr]

In [86]:
con = sqlite3.connect("irs_migration_state.sqlite")

cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
results_name = cursor.fetchall()
results_name1 = results_name[18:-1]
for i in range(len(results_name1)):
    results_name1[i] = str(results_name1[i])[2:-3]
results_name1.sort()
for i in range(int(len(results_name1)/2)):
    #print(results_name1[2*i])
    df = pd.read_sql_query("SELECT * from " + results_name1[2*i], con)
    if i>= int(len(results_name1)/4):
        df = df.set_index('st_dest_name')
        nonmig_df = df[df.index.str.contains("Migrant") == True]
        nonmig_df = nonmig_df[nonmig_df['st_dest_abbrv'].str.contains("FR")== False]
        #mig_table_nonmig.iloc[:10,19 + int(i-len(results_name1)/4)] = nonmig_df.values[:10,5]
        #mig_table_nonmig.iloc[10:,19 + int(i-len(results_name1)/4)] = nonmig_df.values[11:,5]
        if nonmig_df.shape[0] == 51:
            mig_table_nonmig.iloc[:,int(19 + int(i-len(results_name1)/4))] = nonmig_df.values[:,5]
    else:
        df = df.set_index('st_orig_name')
        nonmig_df = df[df.index.str.contains("Migrant") == True]
        nonmig_df = nonmig_df[nonmig_df['st_dest_abbrv'].str.contains("FR")== False]
        #mig_table_nonmig.iloc[:10,19 + int(i-len(results_name1)/4)] = nonmig_df.values[:10,5]
        #mig_table_nonmig.iloc[10:,19 + int(i-len(results_name1)/4)] = nonmig_df.values[11:,5]
        if nonmig_df.shape[0] == 51:
            mig_table_nonmig.iloc[:,int(19 + i)] = nonmig_df.values[:,5]
for i in range(int(len(results_name1)/2)):
    #print(results_name1[2*i+1])
    df = pd.read_sql_query("SELECT * from " + results_name1[2*i + 1], con)
    if i>= int(len(results_name1)/4):
        df = df.set_index('st_orig_abbrv')
        total_mig = df[df.index.str.contains("Migration") == True ].iloc[0::3,5]
        
        if total_mig.shape[0] ==0:
            total_mig = df[df['st_dest_abbrv'].str.contains("FR")== False].iloc[:,5]
        if total_mig.shape[0] >51:
            total_mig = df[df['st_dest_abbrv'].str.contains("FR")== False].iloc[0::3,5]
        if total_mig.shape[0] == 51:
            total_mig = total_mig.reindex(State_name_index_abbr)
            mig_table_outflow.iloc[:,int(19 + int(i-len(results_name1)/4))] = total_mig.values
        else:
            1#print('outflow') 
    else:
        df = df.set_index('st_orig_abbrv')
        total_mig = df[df.index.str.contains("Migration") == True].iloc[0::3,5]
        if total_mig.shape[0] ==0:
            total_mig = df[df['st_dest_abbrv'].str.contains("FR")== False].iloc[:,5]
        if total_mig.shape[0] >51:
            total_mig = df[df['st_dest_abbrv'].str.contains("FR")== False].iloc[0::3,5]
        if total_mig.shape[0] == 51:
            total_mig = total_mig.reindex(State_name_index_abbr)
            mig_table_inflow.iloc[:,int(19 + i)] = total_mig.values
        else:
            1#print('inflow')    
#df = pd.read_sql_table(results_name[18], con)
con.close()

In [87]:
con = sqlite3.connect("irs_migration_state.sqlite")

outflow_1997_98 = pd.read_sql_query("SELECT * from outflow_1997_98", con)
inflow_2002_03 = pd.read_sql_query("SELECT * from inflow_2002_03", con)
outflow_1993_94 = pd.read_sql_query("SELECT * from outflow_1993_94", con)
outflow_1995_96 = pd.read_sql_query("SELECT * from outflow_1995_96", con)
outflow_2013_14 = pd.read_sql_query("SELECT * from outflow_2013_14", con)
outflow_2014_15 = pd.read_sql_query("SELECT * from outflow_2014_15", con)
con.close()

In [88]:
outflow_1997_98_a = outflow_1997_98.pivot_table(index = 'st_orig_abbrv' , columns = 'st_dest_name', values = 'returns')
outflow_1997_98_b = outflow_1997_98_a.iloc[:,outflow_1997_98_a.columns.str.contains("Migrants") == False]
outflow_1997_98_c = outflow_1997_98_b.reindex(State_abbrev.iloc[3:54,3].values)
mig_table_outflow.iloc[:,28] = outflow_1997_98_c.fillna(0).values.sum(1)
#take averages between years for the missing data
for i in range(51):
    if (mig_table_outflow.iloc[i,28] == 0):
        mig_table_outflow.iloc[i,28] = int(mig_table_outflow.iloc[i,27] + mig_table_outflow.iloc[i,29])/2

In [89]:
inflow_2002_03_a = inflow_2002_03.pivot_table(index = 'st_dest_abbrv' , columns = 'st_orig_name', values = 'returns')
inflow_2002_03_b = inflow_2002_03_a.iloc[:,inflow_2002_03_a.columns.str.contains("Migrants") == False]
inflow_2002_03_c = inflow_2002_03_b.reindex(State_abbrev.iloc[3:54,3].values)
mig_table_inflow.iloc[:,33] = inflow_2002_03_c.fillna(0).values.sum(1)
#take averages between years for the missing data
for i in range(51):
    if (mig_table_inflow.iloc[i,33] == 0):
        mig_table_inflow.iloc[i,33] = int(mig_table_inflow.iloc[i,32] + mig_table_inflow.iloc[i,34])/2

In [90]:
outflow_1993_94_a = outflow_1993_94.pivot_table(index = 'st_orig_abbrv' , columns = 'st_dest_abbrv', values = 'returns')
outflow_1993_94_b = outflow_1993_94_a.iloc[outflow_1993_94_a.index.str.contains("FR") == False,outflow_1993_94_a.columns.str.contains("FR") == False]
outflow_1993_94_c = outflow_1993_94_b.reindex(State_abbrev.iloc[3:54,3].values)
outflow_1993_94_c = outflow_1993_94_c.reindex(State_abbrev.iloc[3:54,3].values,axis =1)
outflow_1993_94_c = outflow_1993_94_c.fillna(0)
mig_table_nonmig.iloc[:,24] = outflow_1993_94_c.values.diagonal()
for i in range(51):
    if (mig_table_nonmig.iloc[i,24] == 0):
        mig_table_nonmig.iloc[i,24] = int(mig_table_nonmig.iloc[i,23] + mig_table_nonmig.iloc[i,25])/2

In [91]:
outflow_1995_96_a = outflow_1995_96.pivot_table(index = 'st_orig_abbrv' , columns = 'st_dest_abbrv', values = 'returns')
outflow_1995_96_b = outflow_1995_96_a.iloc[outflow_1995_96_a.index.str.contains("FR") == False,outflow_1995_96_a.columns.str.contains("FR") == False]
outflow_1995_96_c = outflow_1995_96_b.reindex(State_abbrev.iloc[3:54,3].values)
outflow_1995_96_c = outflow_1995_96_c.reindex(State_abbrev.iloc[3:54,3].values,axis =1)
outflow_1995_96_c = outflow_1995_96_c.fillna(0)
mig_table_nonmig.iloc[:,26] = outflow_1995_96_c.values.diagonal()

In [92]:
outflow_2013_14_a = outflow_2013_14.pivot_table(index = 'st_orig_abbrv' , columns = 'st_dest_name', values = 'returns')
outflow_2013_14_b = outflow_2013_14_a.iloc[:,outflow_2013_14_a.columns.str.contains("Non-migrants") == True]
outflow_2013_14_c = outflow_2013_14_b.reindex(State_abbrev.iloc[3:54,3].values)
mig_table_nonmig.iloc[:,-3] = outflow_2013_14_c.fillna(0).values.sum(1)

In [93]:
outflow_2014_15_a = outflow_2014_15.pivot_table(index = 'st_orig_abbrv' , columns = 'st_dest_name', values = 'returns')
outflow_2014_15_b = outflow_2014_15_a.iloc[:,outflow_2014_15_a.columns.str.contains("Non-migrants") == True]
outflow_2014_15_c = outflow_2014_15_b.reindex(State_abbrev.iloc[3:54,3].values)
mig_table_nonmig.iloc[:,-2] = outflow_2014_15_c.fillna(0).values.sum(1)

In [94]:
mig_table_nonmig1 =  mig_table_nonmig.replace(0,1)
mig_table_outflow_rate = mig_table_outflow/mig_table_nonmig1.values
mig_table_inflow_rate = mig_table_inflow/mig_table_nonmig1.values
mig_table_netflow = mig_table_inflow_rate - mig_table_outflow_rate.values
mig_table_nonmig1_rate = mig_table_nonmig1 / mig_table_nonmig1.values.sum(0)

Regional migration from the census bureau

In [95]:
df = pd.read_excel('https://www2.census.gov/programs-surveys/demo/tables/geographic-mobility/time-series/historic/tab-a-2.xls',header = 4)

In [96]:
df['Northeast'] =  df['Northeast'].astype(str).str.replace('*','').str.replace(',','')
df['Midwest'] =  df['Midwest'].astype(str).str.replace('*','').str.replace(',','')
df['South'] =  df['South'].astype(str).str.replace('*','').str.replace(',','')
df['West'] =  df['West'].astype(str).str.replace('*','').str.replace(',','')
df = df.sort_index(axis=0 ,ascending=False)

In [97]:
regional_mig_table_netflow = state_consumption.copy()
as_list = regional_mig_table_netflow.index.tolist()
as_list[0] = 'NORTHEAST'
as_list[1] = 'MIDWEST'
as_list[2] = 'SOUTH'
as_list[3] = 'WEST'
regional_mig_table_netflow.index = as_list
regional_mig_table_netflow = regional_mig_table_netflow.drop(regional_mig_table_netflow.index[4:])
regional_mig_table_netflow.iloc[:,:] = 0 

In [98]:
regional_mig_table_netflow.iloc[:,11:25] = df.iloc[11:105:7,1:].applymap(float).values.T
int_migr95_on = df.iloc[111:-5:7,1:]
int_migr95_on = int_migr95_on.reset_index()
int_migr95_on = int_migr95_on.drop([5,6,16,18])
int_migr95_on = int_migr95_on.iloc[:,1:]
regional_mig_table_netflow.iloc[:,26:] = int_migr95_on.applymap(float).values.T
regional_mig_table_netflow.iloc[:,25] = (regional_mig_table_netflow.iloc[:,24].values +
                                         regional_mig_table_netflow.iloc[:,26].values)/2

Original regressions

In [99]:
wls = 0
rng = list(range(0,51))#list(range(0,7)) + list(range(9, 28)) + list(range(29, 51)) #
no_state = len(rng)
time_length = 46
start_period = 0
original_data = 0 # set to 1 to use ASY original data with Population and CPI from here -rerun the whole code to restore
#2 to use Census regions 
#3 to use European data
migration = 0 # set to 1 to fix population_ratio at the initial ratio - upper bound, incorporates demographics too
employment = 0 # set to 1 for using employment instead of population

In [100]:
if (employment == 1 and (original_data == 0 or original_data == 1 or original_data == 2)):
    Population1 = Population.copy()
    Population = Employment.copy()

In [101]:
Population_star = Population.copy()
if (migration == 1):
    Total_pop = Population_star.values.sum(0)
    for i in range(Population_star.shape[1]):
        Population_star.iloc[:,i] = Population_star.iloc[:,0]/Population_star.iloc[:,0].values.sum() * Total_pop[i]
elif(migration ==2 and original_data == 0 ):
    for i in range(Population_star.shape[1]):
        Population_star.iloc[:,i] = Population_star.iloc[:,i] * (1 - mig_table_netflow.values[:,i])



In [102]:
if (original_data == 1):
    non_federal_state_income = pd.read_excel('nch6.xls',header =1)
    p = non_federal_state_income.pivot(index = 'State' , columns = 'Region')
    non_federal_state_income = p.stack()
    non_federal_state_income = non_federal_state_income.reset_index(level=1, drop=True)
    non_federal_state_income  = non_federal_state_income.iloc[:,:-1]
    disposable_state_income = pd.read_excel('nch11.xls',header =1)
    p = disposable_state_income.pivot(index = 'State' , columns = 'Region')
    disposable_state_income = p.stack()
    disposable_state_income = disposable_state_income.reset_index(level=1, drop=True)
    disposable_state_income  = disposable_state_income.iloc[:,:]
    nominal_Industry_Total_GSP1 = pd.read_excel('nch14.xls',header =1)
    nominal_Industry_Total_GSP1 = nominal_Industry_Total_GSP1.drop(nominal_Industry_Total_GSP1.index[-1])
    p = nominal_Industry_Total_GSP1.pivot(index = 'State' , columns = 'Region')
    nominal_Industry_Total_GSP1 = p.stack()
    nominal_Industry_Total_GSP1 = nominal_Industry_Total_GSP1.reset_index(level=1, drop=True)
    nominal_Industry_Total_GSP1 = nominal_Industry_Total_GSP1.iloc[:,:-1]
    state_consumption = pd.read_excel('nch15.xls',header =1)
    p = state_consumption.pivot(index = 'State' , columns = 'Region')
    state_consumption = p.stack()
    state_consumption = state_consumption.reset_index(level=1, drop=True)
    state_consumption  = state_consumption.iloc[:,:]
    names = Industry_Total_GSP.index
    names_vector2 = [s + ' state total' for s in names] + [s for s in names] + [s + '*' for s in names] 
    SA4_table3 = SA4_table1.loc[names_vector2]
    Population = SA4_table3.iloc[20::22].applymap(float)
    Population = Population.iloc[:,:36]
    cpi = GDP_deflator_annual.iloc[16:52,0].values
    cpi_tile = np.tile(cpi,(no_state,1))
    cpi_flat = cpi_tile.T.flatten()/ 100
    time_length = 26
    start_period = 0
    pe_cons = state_consumption.iloc[rng,:]/np.sqrt(Population.iloc[rng,:].values)/cpi_tile * 1000000
    pe_gsp = nominal_Industry_Total_GSP1.iloc[rng,:]/np.sqrt(Population.iloc[rng,:].values)/cpi_tile * 1000000
    pe_income = non_federal_state_income.iloc[rng,:]/np.sqrt(Population.iloc[rng,:].values)/cpi_tile * 1000000
    pe_disp_income = disposable_state_income.iloc[rng,:]/np.sqrt(Population.iloc[rng,:].values) /cpi_tile* 1000000
    pe_cons = pe_cons.astype(float)
    pe_gsp = pe_gsp.astype(float)
    pe_income = pe_income.astype(float)
    pe_disp_income = pe_disp_income.astype(float)
    tostata_save = pe_cons.unstack().to_frame()
    tostata_save.columns = ['Consumption']
    tostata_save['GSP'] = pe_gsp.unstack().values
    tostata_save['Inc'] = pe_income.unstack().values.T.flatten()
    tostata_save['DInc'] = pe_disp_income.unstack().values.T.flatten()
    tostata_save.to_csv('state.csv')
    pe_cons = pe_cons.values.T.flatten()
    pe_gsp = pe_gsp.values.T.flatten()
    pe_income = pe_income.values.T.flatten()
    pe_disp_income = pe_disp_income.values.T.flatten()
elif(original_data == 2):
    #Census bureau regional aggregation
    NORTHEAST_index = np.asarray((6,19,21,29,30,32,38,39,45))
    MIDWEST_index = np.asarray((13,14,15,16,22,23,27,34,35,41,49))
    SOUTH_index = np.asarray((0,3,7,8,9,10,17,18,20,24,25,33,36,40,42,43,46,48))
    WEST_index = np.asarray((2,4,5,12,26,28,31,37,44,47,50))
    rng = list(range(0,4))
    no_state = len(rng)
    cpi = GDP_deflator_annual.iloc[22:69,0].values/ GDP_deflator_annual.iloc[43,0]
    cpi_tile = np.tile(cpi,(no_state,1))
    cpi_flat = cpi_tile.T.flatten()
    region_cons = state_consumption.copy()
    as_list = region_cons.index.tolist()
    as_list[0] = 'NORTHEAST'
    as_list[1] = 'MIDWEST'
    as_list[2] = 'SOUTH'
    as_list[3] = 'WEST'
    region_cons.index = as_list
    region_cons.loc['NORTHEAST',:] = state_consumption.iloc[NORTHEAST_index,:].values.sum(0)
    region_cons.loc['MIDWEST',:] = state_consumption.iloc[MIDWEST_index,:].values.sum(0)
    region_cons.loc['SOUTH',:] = state_consumption.iloc[SOUTH_index,:].values.sum(0)
    region_cons.loc['WEST',:] = state_consumption.iloc[WEST_index,:].values.sum(0)
    region_cons = region_cons.drop(region_cons.index[4:])
    nominal_Industry_Total_GSP_regional = region_cons.copy() 
    nominal_Industry_Total_GSP_regional.loc['NORTHEAST',:] = nominal_Industry_Total_GSP1.iloc[NORTHEAST_index,:].values.sum(0)
    nominal_Industry_Total_GSP_regional.loc['MIDWEST',:] = nominal_Industry_Total_GSP1.iloc[MIDWEST_index,:].values.sum(0)
    nominal_Industry_Total_GSP_regional.loc['SOUTH',:] = nominal_Industry_Total_GSP1.iloc[SOUTH_index,:].values.sum(0)
    nominal_Industry_Total_GSP_regional.loc['WEST',:] = nominal_Industry_Total_GSP1.iloc[WEST_index,:].values.sum(0)
    non_federal_state_income_regional = region_cons.copy() 
    non_federal_state_income_regional.loc['NORTHEAST',:] = non_federal_state_income.iloc[NORTHEAST_index,:].values.sum(0)
    non_federal_state_income_regional.loc['MIDWEST',:] = non_federal_state_income.iloc[MIDWEST_index,:].values.sum(0)
    non_federal_state_income_regional.loc['SOUTH',:] = non_federal_state_income.iloc[SOUTH_index,:].values.sum(0)
    non_federal_state_income_regional.loc['WEST',:] = non_federal_state_income.iloc[WEST_index,:].values.sum(0)
    disposable_state_income_regional = region_cons.copy() 
    disposable_state_income_regional.loc['NORTHEAST',:] = disposable_state_income.iloc[NORTHEAST_index,:].values.sum(0)
    disposable_state_income_regional.loc['MIDWEST',:] = disposable_state_income.iloc[MIDWEST_index,:].values.sum(0)
    disposable_state_income_regional.loc['SOUTH',:] = disposable_state_income.iloc[SOUTH_index,:].values.sum(0)
    disposable_state_income_regional.loc['WEST',:] = disposable_state_income.iloc[WEST_index,:].values.sum(0)
    Population_regional = region_cons.copy()
    
    Population_regional.loc['NORTHEAST',:] = Population.iloc[NORTHEAST_index,:].values.sum(0)
    Population_regional.loc['MIDWEST',:] = Population.iloc[MIDWEST_index,:].values.sum(0)
    Population_regional.loc['SOUTH',:] = Population.iloc[SOUTH_index,:].values.sum(0)
    Population_regional.loc['WEST',:] = Population.iloc[WEST_index,:].values.sum(0)
    Population_star_regional = Population_regional.copy()
    if (migration == 1):
        Total_pop = Population_star_regional.values.sum(0)
        for i in range(Population_star_regional.shape[1]):
            Population_star_regional.iloc[:,i] = Population_star_regional.iloc[:,0]/Population_star_regional.iloc[:,0].values.sum() * Total_pop[i]
    if(migration ==2):
        for i in range(Population_star.shape[1]):
            Population_star_regional.iloc[:,i] = Population_star_regional.iloc[:,i] - regional_mig_table_netflow.values[:,i]*1000
    if wls == 0:
        pe_cons = region_cons.iloc[rng,:].values.T.flatten()/(Population_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_gsp = nominal_Industry_Total_GSP_regional.iloc[rng,:].values.T.flatten()/(Population_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_gsp_star = nominal_Industry_Total_GSP_regional.iloc[rng,:].values.T.flatten()/(Population_star_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_income =  non_federal_state_income_regional.iloc[rng,:].values.T.flatten()/(Population_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_disp_income = disposable_state_income_regional.iloc[rng,:].values.T.flatten()/(Population_regional.iloc[rng,:].values.T.flatten()) /cpi_flat* 1000000
        mu_gr = Population_regional.iloc[rng,:].values.T.flatten()/(Population_star_regional.iloc[rng,:].values.T.flatten())
    else:
        pe_cons = region_cons.iloc[rng,:].values.T.flatten()/np.sqrt(Population_regional.iloc[rng,:].values.T.flatten()) * 1000000 /cpi_flat
        pe_gsp = nominal_Industry_Total_GSP_regional.iloc[rng,:].values.T.flatten()/np.sqrt(Population_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_gsp_star = nominal_Industry_Total_GSP_regional.iloc[rng,:].values.T.flatten()/np.sqrt(Population_star_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_income =  non_federal_state_income_regional.iloc[rng,:].values.T.flatten()/np.sqrt(Population_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_disp_income = disposable_state_income_regional.iloc[rng,:].values.T.flatten()/np.sqrt(Population_regional.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        mu_gr = Population_regional.iloc[rng,:].values.T.flatten()/np.sqrt(Population_star_regional.iloc[rng,:].values.T.flatten())
elif(original_data == 3):
    EU_data_gdp = pd.read_csv('SNA_TABLE2_gdp.csv',header =0)
    EU_data_pop = pd.read_csv('SNA_TABLE3_pop.csv',header =0)
    EU_data_migr = pd.read_csv('EU_data_migration1.csv',header =5)
    no_EU_members = 13
    no_state = no_EU_members
    rng = list(range(0,no_EU_members))
    EU_data_gdp1 = EU_data_gdp.iloc[:,[1,3,7,14]]
    EU_data_pop1 = EU_data_pop.iloc[:,[1,3,7,14]]
    EU_data_gdp2 = EU_data_gdp1.pivot_table(values = 'Value', index=['Transaction','Country'],columns = 'Year')
    EU_data_pop2 = EU_data_pop1.pivot_table(values = 'Value', index=['Transaction','Country'],columns = 'Year')
    EU_cons = EU_data_gdp2.iloc[:no_EU_members,:].unstack().unstack()
    EU_cons.index = EU_cons.index.droplevel(level=2)
    EU_cons = EU_cons.to_frame().pivot_table(columns = 'Year',index = 'Country')
    EU_cons.columns = EU_cons.columns.droplevel(level=0)
    EU_GDP = EU_data_gdp2.iloc[no_EU_members:(2*no_EU_members),:].unstack().unstack()
    EU_GDP.index = EU_GDP.index.droplevel(level=2)
    EU_GDP = EU_GDP.to_frame().pivot_table(columns = 'Year',index = 'Country')
    EU_GDP.columns = EU_GDP.columns.droplevel(level=0)
    EU_GNDI = EU_data_gdp2.iloc[(2*no_EU_members):(3*no_EU_members),:].unstack().unstack()
    EU_GNDI.index = EU_GNDI.index.droplevel(level=2)
    EU_GNDI = EU_GNDI.to_frame().pivot_table(columns = 'Year',index = 'Country')
    EU_GNDI.columns = EU_GNDI.columns.droplevel(level=0)
    EU_GNI = EU_data_gdp2.iloc[(3*no_EU_members):(4*no_EU_members),:].unstack().unstack()
    EU_GNI.index = EU_GNI.index.droplevel(level=2)
    EU_GNI = EU_GNI.to_frame().pivot_table(columns = 'Year',index = 'Country')
    EU_GNI.columns = EU_GNI.columns.droplevel(level=0)
    EU_employment = EU_data_pop2.iloc[:no_EU_members,:].unstack().unstack()
    EU_employment.index = EU_employment.index.droplevel(level=2)
    EU_employment = EU_employment.to_frame().pivot_table(columns = 'Year',index = 'Country')
    EU_employment.columns = EU_employment.columns.droplevel(level=0)
    EU_pop = EU_data_pop2.iloc[no_EU_members:(2*no_EU_members),:].unstack().unstack()
    EU_pop.index = EU_pop.index.droplevel(level=2)
    EU_pop = EU_pop.to_frame().pivot_table(columns = 'Year',index = 'Country')
    EU_pop.columns = EU_pop.columns.droplevel(level=0)
    EU_pop_star = EU_pop.copy()
    EU_data_migr = EU_data_migr.drop([EU_data_migr.index[14],EU_data_migr.index[0]])
    EU_data_migr = EU_data_migr.drop(EU_data_migr.columns[1],axis =1)
    EU_data_migr = EU_data_migr.set_index(['Year'])
    EU_data_migr = EU_data_migr.replace('..',np.nan)
    EU_data_migr = EU_data_migr.applymap(float)
    EU_data_migr_rate = EU_data_migr / EU_pop.iloc[:,14:].values/1000
    EU_pop1 = EU_pop.iloc[:,14:] * (np.isnan(EU_data_migr_rate.values )== 0 * np.ones(EU_data_migr_rate.values.shape))
    EU_pop2 = EU_pop1 / EU_pop1.sum()
    if (migration == 1):
        Total_pop = EU_pop_star.values.sum(0)
        for i in range(EU_pop_star.shape[1]):
            EU_pop_star.iloc[:,i] = EU_pop_star.iloc[:,0]/EU_pop_star.iloc[:,0].values.sum() * Total_pop[i]
    cpi = GDP_deflator_annual.iloc[23:69,0].values/ GDP_deflator_annual.iloc[43,0]
    cpi_tile = np.tile(cpi,(no_EU_members,1))
    cpi_flat = cpi_tile.T.flatten()
    if wls == 0:
        pe_cons = EU_cons.iloc[rng,:].values.T.flatten()/(EU_pop.iloc[rng,:].values.T.flatten())/cpi_flat * 1000
        pe_gsp = EU_GDP.iloc[rng,:].values.T.flatten()/(EU_pop.iloc[rng,:].values.T.flatten())/cpi_flat * 1000
        pe_gsp_star = EU_GDP.iloc[rng,:].values.T.flatten()/(EU_pop_star.iloc[rng,:].values.T.flatten())/cpi_flat * 1000
        pe_income =  EU_GNI.iloc[rng,:].values.T.flatten()/(EU_pop.iloc[rng,:].values.T.flatten())/cpi_flat * 1000
        pe_disp_income = EU_GNDI.iloc[rng,:].values.T.flatten()/(EU_pop.iloc[rng,:].values.T.flatten()) /cpi_flat* 1000
        mu_gr = EU_pop.iloc[rng,:].values.T.flatten()/(EU_pop_star.iloc[rng,:].values.T.flatten())
    else:
        pe_cons = EU_cons.iloc[rng,:].values.T.flatten()/np.sqrt(EU_pop.iloc[rng,:].values.T.flatten()) * 1000 /cpi_flat
        pe_gsp = EU_GDP.iloc[rng,:].values.T.flatten()/np.sqrt(EU_pop.iloc[rng,:].values.T.flatten())/cpi_flat * 1000
        pe_gsp_star = EU_GDP.iloc[rng,:].values.T.flatten()/np.sqrt(EU_pop_star.iloc[rng,:].values.T.flatten())/cpi_flat * 1000
        pe_income =  EU_GNI.iloc[rng,:].values.T.flatten()/np.sqrt(EU_pop.iloc[rng,:].values.T.flatten())/cpi_flat * 1000
        pe_disp_income = EU_GNDI.iloc[rng,:].values.T.flatten()/np.sqrt(EU_pop.iloc[rng,:].values.T.flatten())/cpi_flat * 1
        mu_gr = EU_pop.iloc[rng,:].values.T.flatten()/np.sqrt(EU_pop_star.iloc[rng,:].values.T.flatten())
else:
    cpi = GDP_deflator_annual.iloc[22:69,0].values/ GDP_deflator_annual.iloc[43,0]
    cpi_tile = np.tile(cpi,(no_state,1))
    cpi_flat = cpi_tile.T.flatten()
    if wls == 0:
        pe_cons = state_consumption.iloc[rng,:].values.T.flatten()/(Population.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_gsp = nominal_Industry_Total_GSP1.iloc[rng,:].values.T.flatten()/(Population.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_gsp_star = nominal_Industry_Total_GSP1.iloc[rng,:].values.T.flatten()/(Population_star.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_income = non_federal_state_income.iloc[rng,:].values.T.flatten()/(Population.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_disp_income = disposable_state_income.iloc[rng,:].values.T.flatten()/(Population.iloc[rng,:].values.T.flatten()) /cpi_flat* 1000000
        mu_gr = Population.iloc[rng,:].values.T.flatten()/(Population_star.iloc[rng,:].values.T.flatten())
    else:
        pe_cons = state_consumption.iloc[rng,:].values.T.flatten()/np.sqrt(Population.iloc[rng,:].values.T.flatten()) * 1000000 /cpi_flat
        pe_gsp = nominal_Industry_Total_GSP1.iloc[rng,:].values.T.flatten()/np.sqrt(Population.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_gsp_star = nominal_Industry_Total_GSP1.iloc[rng,:].values.T.flatten()/np.sqrt(Population_star.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_income = non_federal_state_income.iloc[rng,:].values.T.flatten()/np.sqrt(Population.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        pe_disp_income = disposable_state_income.iloc[rng,:].values.T.flatten()/np.sqrt(Population.iloc[rng,:].values.T.flatten())/cpi_flat * 1000000
        mu_gr = np.sqrt(Population.iloc[rng,:].values.T.flatten())/np.sqrt(Population_star.iloc[rng,:].values.T.flatten())

In [103]:
pe_cons = np.log(pe_cons.astype(float))
pe_gsp = np.log(pe_gsp.astype(float))
pe_gsp_star = np.log(pe_gsp_star.astype(float))
pe_income = np.log(pe_income.astype(float))
pe_disp_income = np.log(pe_disp_income.astype(float))
mu_gr = np.log(mu_gr.astype(float))

In [104]:
# Save the data to stata -EU:
#stata_df = pd.DataFrame(data = ((EU_cons/ EU_pop.values)/cpi_flat.reshape(EU_pop.values.shape, order ='F') * 1000).unstack().to_frame())
#stata_df = stata_df.assign(Output = ((EU_GDP/ EU_pop.values)/cpi_flat.reshape(EU_pop.values.shape, order ='F') * 1000).unstack().to_frame().values)
#stata_df = stata_df.assign(Output_star = ((EU_GDP/ EU_pop_star.values)/cpi_flat.reshape(EU_pop.values.shape, order ='F') * 1000).unstack().to_frame().values)
#stata_df = stata_df.assign(Income = ((EU_GNI/ EU_pop.values)/cpi_flat.reshape(EU_pop.values.shape, order ='F') * 1000).unstack().to_frame().values)
#stata_df = stata_df.assign(Disposable_Income = ((EU_GNDI/ EU_pop.values)/cpi_flat.reshape(EU_pop.values.shape, order ='F') * 1000).unstack().to_frame().values)

In [105]:
# Save the data to stata-US state level:
stata_df = pd.DataFrame(data = ((state_consumption.iloc[rng,:]/ Population.iloc[rng,:].values)/cpi_flat.reshape(Population.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame())
stata_df = stata_df.assign(Output = ((nominal_Industry_Total_GSP1.iloc[rng,:]/ Population.iloc[rng,:].values)/cpi_flat.reshape(Population.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)
stata_df = stata_df.assign(Output_star = ((nominal_Industry_Total_GSP1.iloc[rng,:]/ Population_star.iloc[rng,:].values)/cpi_flat.reshape(Population.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)
stata_df = stata_df.assign(Income = ((non_federal_state_income.iloc[rng,:]/ Population.iloc[rng,:].values)/cpi_flat.reshape(Population.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)
stata_df = stata_df.assign(Disposable_Income = ((disposable_state_income.iloc[rng,:]/ Population.iloc[rng,:].values)/cpi_flat.reshape(Population.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [ ]:
# Save the data to stata-US regional level:
#stata_df = pd.DataFrame(data = ((region_cons.iloc[rng,:]/ Population_regional.iloc[rng,:].values)/cpi_flat.reshape(Population_regional.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame())
#stata_df = stata_df.assign(Output = ((nominal_Industry_Total_GSP_regional.iloc[rng,:]/ Population_regional.iloc[rng,:].values)/cpi_flat.reshape(Population_regional.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)
#stata_df = stata_df.assign(Output_star = ((nominal_Industry_Total_GSP_regional.iloc[rng,:]/ Population_star_regional.iloc[rng,:].values)/cpi_flat.reshape(Population_regional.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)
#stata_df = stata_df.assign(Income = ((non_federal_state_income_regional.iloc[rng,:]/ Population_regional.iloc[rng,:].values)/cpi_flat.reshape(Population_regional.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)
#stata_df = stata_df.assign(Disposable_Income = ((disposable_state_income_regional.iloc[rng,:]/ Population_regional.iloc[rng,:].values)/cpi_flat.reshape(Population_regional.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame().values)


In [ ]:
stata_df.columns = ['Consumption','Output','Output_star','Income','Disposable_Income']
stata_df.reset_index(inplace=True)
#stata_df.level_1 = stata_df.level_1.str.encode('latin-1')
#stata_df.level_0 = stata_df.level_0.str.encode('latin-1')
#list(stata_df.select_dtypes(include=['object']).columns)
#stata_df['level_0'] = stata_df['level_0'].astype('int64')
#stata_df['level_1'] = stata_df['level_1'].astype(str)
#stata_df['Consumption'] = stata_df['Consumption'].astype('float')
#stata_df['Output'] = stata_df['Output'].astype('float')
#stata_df['Output_star'] = stata_df['Output_star'].astype('float')
#stata_df['Income'] = stata_df['Income'].astype('float')
#stata_df['Disposable_Income'] = stata_df['Disposable_Income'].astype('float')

In [ ]:
#stata_df.to_stata('EU_data.dta')
stata_df.to_stata('US_state_data.dta')
#stata_df.to_stata('US_regional_data.dta')

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
ax.plot(np.arange(1988,2015),(mig_table_inflow_rate.iloc[:,19:-1] * mig_table_nonmig1_rate.iloc[:,19:-1].values).sum(),color='blue',
        linewidth=3, markersize=12,label = 'US')
ax.plot(np.arange(1988,2015),(EU_data_migr_rate * EU_pop2.values).sum().values[4:-1],color='red',  linestyle='dotted',
        linewidth=3, markersize=12,label = 'EU')
ax.legend(loc="center right",fontsize=24)
tikz_save('migr_rate_EUS.tex', figureheight='\\figureheight',figurewidth='\\figurewidth')
fig.savefig('migr_rate_EUS.png', dpi=100)
plt.show()

In [ ]:
tikz_save?

In [112]:
stata_df

0     Output  Output_star
     state                                                
1969 Alabama               8.68157  10.532550    10.532550
     Alaska                12.1579  19.840106    19.840106
     Arizona               11.0621  13.724303    13.724303
     Arkansas              9.80962  10.017137    10.017137
     California            12.8676  17.321310    17.321310
     Colorado              11.2209  13.708082    13.708082
     Connecticut            11.996  15.975278    15.975278
     Delaware              12.2021  18.684362    18.684362
     District of Columbia  15.2307  33.847921    33.847921
     Florida               11.7348  12.980476    12.980476
     Georgia               10.0715  12.560851    12.560851
     Hawaii                11.9158  16.978427    16.978427
     Idaho                 11.0593  11.619451    11.619451
     Illinois              12.1758  17.485240    17.485240
     Indiana               11.3666  14.916386    14.916386
     Iowa                  12.4263  13.606846    13.606846
     Kansas                10.8974  12.761375    12.761375
     Kentucky              9.24263  12.828080    12.828080
     Louisiana             9.77571  12.966416    12.966416
     Maine                  10.216  11.417021    11.417021
     Maryland              11.3396  14.370104    14.370104
     Massachusetts         11.7533  15.613419    15.613419
     Michigan              11.6692  17.706758    17.706758
     Minnesota             11.4556  14.498990    14.498990
     Mississippi           8.29872   9.180352     9.180352
     Missouri              11.1328  14.420237    14.420237
     Montana               11.3328  12.374870    12.374870
     Nebraska              11.9286  13.698224    13.698224
     Nevada                 14.293  19.037821    19.037821
     New Hampshire         11.3918  12.421336    12.421336
...                            ...        ...          ...
2015 Massachusetts         35.7826  44.118811    44.118811
     Michigan              27.1184  29.098235    29.098235
     Minnesota             30.7587  36.556836    36.556836
     Mississippi           21.9628  21.717540    21.717540
     Missouri              25.8309  29.541550    29.541550
     Montana               29.0189  27.239414    27.239414
     Nebraska              27.9034  36.738823    36.738823
     Nevada                25.1802  29.997983    29.997983
     New Hampshire         33.1579  34.240168    34.240168
     New Jersey            34.3592  38.730883    38.730883
     New Mexico            25.8566  27.481477    27.481477
     New York              33.8418  44.891332    44.891332
     North Carolina        23.7301  30.537660    30.537660
     North Dakota          36.2395  45.314429    45.314429
     Ohio                  26.2614  32.091466    32.091466
     Oklahoma              23.6722  29.505979    29.505979
     Oregon                28.2229  32.990723    32.990723
     Pennsylvania          28.5575  33.959461    33.959461
     Rhode Island          29.4646  32.328020    32.328020
     South Carolina        23.8272  25.285183    25.285183
     South Dakota          27.6563  33.646647    33.646647
     Tennessee              23.519  29.443508    29.443508
     Texas                 25.6495  36.019908    36.019908
     Utah                  24.7024  30.572289    30.572289
     Vermont               33.5712  29.677236    29.677236
     Virginia              29.2081  35.302994    35.302994
     Washington            30.4593  38.231938    38.231938
     West Virginia          24.295  24.442819    24.442819
     Wisconsin              27.652  32.063562    32.063562
     Wyoming                32.539  41.206809    41.206809

[2397 rows x 3 columns]

In [111]:
((non_federal_state_income.iloc[rng,:]/ Population.iloc[rng,:].values)/cpi_flat.reshape(Population.iloc[rng,:].values.shape, order ='F') * 1000).unstack().to_frame()

AttributeError: 'DataFrame' object has no attribute 'to_frame'

Demean data

In [ ]:
def demean(vari):
    ones_vari = np.ones(vari.shape)
    return vari - ones_vari @ np.linalg.solve(ones_vari.T @  ones_vari, ones_vari.T @  vari)

In [ ]:
time_length = 20
start_period = 0
def time_horizon_gls(start_period,time_length):
    def OLS(X,y,Omega_inv = np.eye(time_length  * no_state,time_length  * no_state)):
        beta = np.linalg.solve(X.T @ Omega_inv @  X, X.T @ Omega_inv  @ y)
        y_hat = X @ beta
        var = np.sum((y_hat - y)**2)/(time_length  * no_state - 1 )
        var_beta = var *  np.linalg.inv(X.T @ X)
        return beta , y_hat , var , var_beta
    time_fixed_effects = np.kron(np.eye(time_length,time_length),np.ones((no_state,1)))
    shape_vec = pe_cons[no_state:].shape[0]
    d_pe_cons = np.reshape(pe_cons[no_state:] - pe_cons[:-no_state],(shape_vec,1))
    d_pe_gsp = np.reshape(pe_gsp[no_state:] - pe_gsp[:-no_state],(shape_vec,1))
    d_pe_income = np.reshape(pe_income[no_state:] - pe_income[:-no_state],(shape_vec,1))
    d_pe_disp_income = np.reshape(pe_disp_income[no_state:] - pe_disp_income[:-no_state],(shape_vec,1))
    d_pe_cons = demean(d_pe_cons)
    d_pe_gsp = demean(d_pe_gsp)
    d_pe_income = demean(d_pe_income)
    d_pe_disp_income = demean(d_pe_disp_income)
    y_1 =  d_pe_income[(start_period * no_state):((time_length + start_period)  * no_state)]
    X = d_pe_gsp[(start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,X_mean,var,var_beta = OLS(time_fixed_effects,X)
    X = X - X_mean
    X = demean(X)
    y_1 = demean(y_1)
    beta_mean,y_1_mean,var,var_beta = OLS(time_fixed_effects,y_1)
    y_1 = y_1 - y_1_mean
    beta , y_1_hat , var , var_beta = OLS(X,y_1)
    beta_capital = beta[0]
    beta_capital,np.sqrt(var_beta[0,0])
    res_1 = y_1_hat - y_1
    w_1 = res_1**2
    w_1_diag = w_1.copy()
    for i in range(no_state):
        w_1_diag[i::no_state] =  np.sqrt(1/(w_1[i::no_state].mean()))
    Omega_inv = np.diagflat(w_1_diag)
    X_1 = Omega_inv @ X
    y_1g = Omega_inv @ y_1
    res_1g = Omega_inv @ res_1
    #Federal redistribution
    y_2 =  d_pe_disp_income[
       (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_2_mean,var,var_beta = OLS(time_fixed_effects,y_2)
    y_2 = y_2 - y_2_mean
    y_2 = demean(y_2)
    beta , y_2_hat , var , var_beta = OLS(X,y_2)
    beta_federal = beta[0]
    beta_federal,np.sqrt(var_beta[0,0])
    res_2 = y_2_hat - y_2
    w_2 = res_2**2
    w_2_diag = w_2.copy()
    for i in range(no_state):
        w_2_diag[i::no_state] =  np.sqrt(1/(w_2[i::no_state].mean()))
    Omega_inv = np.diagflat(w_2_diag)
    X_2 = Omega_inv @ X
    y_2g = Omega_inv @ y_2
    res_2g = Omega_inv @ res_2
    #Credit market
    y_3 =  d_pe_cons[
        (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_3_mean,var,var_beta = OLS(time_fixed_effects,y_3)
    y_3 = y_3 - y_3_mean
    y_3 = demean(y_3)
    beta , y_3_hat , var , var_beta = OLS(X,y_3)
    beta_credit = beta[0]
    beta_credit,np.sqrt(var_beta[0,0])
    res_3 = y_3_hat - y_3
    w_3 = res_3**2
    w_3_diag = w_3.copy()
    for i in range(no_state):
        w_3_diag[i::no_state] =  np.sqrt(1/(w_3[i::no_state].mean()))
    Omega_inv = np.diagflat(w_3_diag)
    X_3 = Omega_inv @ X
    y_3g = Omega_inv @ y_3
    res_3g = Omega_inv @ res_3
    #FGLS
    X_gls = sp.linalg.block_diag(X_1,X_2,X_3)
    y_stack = np.concatenate((np.concatenate((y_1g,y_2g),axis =0),y_3g),axis =0)
    Omega_sur = np.ones((3,3))
    Omega_sur[0,1] = np.mean(res_1g * res_2g) 
    Omega_sur[0,2] = np.mean(res_1g * res_3g) 
    Omega_sur[1,2] = np.mean(res_3g * res_2g) 
    Omega_sur[2,1] = Omega_sur[1,2]
    Omega_sur[2,0] = Omega_sur[0,2]
    Omega_sur[1,0] = Omega_sur[0,1]
    Omega_sur_inv = sp.linalg.inv(Omega_sur)
    Omega_inv = np.kron(Omega_sur_inv, np.eye(time_length  * no_state,time_length  * no_state))
    beta , y_3_hat , var , var_beta = OLS(X_gls,y_stack,Omega_inv)
    eta = np.zeros((4,3))
    eta[0,0] = -1
    eta[1,0] = 1
    eta[1,1] = -1
    eta[2,1] = 1
    eta[2,2] = -1
    eta[3,2] = 1
    var_eta = np.sqrt(np.diag(eta @ var_beta @ eta.T))
    eta = eta @ beta
    eta[0] = eta[0] +1
    return eta,var_eta

In [ ]:
time_horizon_gls(start_period,35)

In [ ]:
start_period_grid = 45 - time_length
smoothing_grid = np.ones((4,start_period_grid))
time_grid = np.ones((start_period_grid,1))
for i in range(start_period_grid):
    time_grid[i,0] = i + 1970 + int(time_length/2)
    coeff,var = time_horizon_gls(i,time_length)
    smoothing_grid[:,i] = coeff[:,0]

In [ ]:
plt.plot(time_grid,smoothing_grid.T)

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["b", "g", "r", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Capital","Federal","Credit","Unsmoothed")
for i in np.arange(0, data_shape[0]):
    ax.bar(time_grid[:,0], data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
ax.legend(loc="best",fontsize=10)
fig.savefig('region.png', dpi=100)
plt.show()

In [ ]:
if (employment == 1 and (original_data == 0 or original_data == 1)):
    Population = Population1.copy()

In [ ]:
def time_horizon_gls4(start_period,time_length):
    def OLS(X,y,Omega_inv = np.eye(time_length  * no_state,time_length  * no_state)):
        beta = np.linalg.solve(X.T @ Omega_inv @  X, X.T @ Omega_inv  @ y)
        y_hat = X @ beta
        var = np.sum((y_hat - y)**2)/(time_length  * no_state - 1 )
        var_beta = var *  np.linalg.inv(X.T @ X)
        return beta , y_hat , var , var_beta
    time_fixed_effects = np.kron(np.eye(time_length,time_length),np.ones((no_state,1)))
    shape_vec = pe_cons[no_state:].shape[0]
    d_pe_cons = np.reshape(pe_cons[no_state:] - pe_cons[:-no_state],(shape_vec,1))
    d_pe_gsp = np.reshape(pe_gsp[no_state:] - pe_gsp[:-no_state],(shape_vec,1))
    d_pe_gsp_star = np.reshape(pe_gsp_star[no_state:] - pe_gsp_star[:-no_state],(shape_vec,1))
    d_pe_income = np.reshape(pe_income[no_state:] - pe_income[:-no_state],(shape_vec,1))
    d_pe_disp_income = np.reshape(pe_disp_income[no_state:] - pe_disp_income[:-no_state],(shape_vec,1))
    d_mu_gr = np.reshape(mu_gr[no_state:] - mu_gr[:-no_state],(shape_vec,1))
    
    d_pe_cons = demean(d_pe_cons)
    d_pe_gsp = demean(d_pe_gsp)
    d_pe_gsp_star = demean(d_pe_gsp_star)
    d_pe_income = demean(d_pe_income)
    d_pe_disp_income = demean(d_pe_disp_income)
    d_mu_gr = demean(d_mu_gr)
    
    y_1 =  d_pe_income[(start_period * no_state):((time_length + start_period)  * no_state)]
    X = d_pe_gsp_star[(start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,X_mean,var,var_beta = OLS(time_fixed_effects,X)
    X = X - X_mean
    X = demean(X)
    y_1 = demean(y_1)
    beta_mean,y_1_mean,var,var_beta = OLS(time_fixed_effects,y_1)
    y_1 = y_1 - y_1_mean
    beta , y_1_hat , var , var_beta = OLS(X,y_1)
    res_1 = y_1_hat - y_1
    w_1 = res_1**2
    w_1_diag = w_1.copy()
    for i in range(no_state):
        w_1_diag[i::no_state] =  np.sqrt(1/(w_1[i::no_state].mean()))
    Omega_inv = np.diagflat(w_1_diag)
    X_1 = Omega_inv @ X
    y_1g = Omega_inv @ y_1
    res_1g = Omega_inv @ res_1
    #Federal redistribution
    y_2 =  d_pe_disp_income[
       (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_2_mean,var,var_beta = OLS(time_fixed_effects,y_2)
    y_2 = y_2 - y_2_mean
    y_2 = demean(y_2)
    beta , y_2_hat , var , var_beta = OLS(X,y_2)
    res_2 = y_2_hat - y_2
    w_2 = res_2**2
    w_2_diag = w_2.copy()
    for i in range(no_state):
        w_2_diag[i::no_state] =  np.sqrt(1/(w_2[i::no_state].mean()))
    Omega_inv = np.diagflat(w_2_diag)
    X_2 = Omega_inv @ X
    y_2g = Omega_inv @ y_2
    res_2g = Omega_inv @ res_2
    #Credit market
    y_3 =  d_pe_cons[
        (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_3_mean,var,var_beta = OLS(time_fixed_effects,y_3)
    y_3 = y_3 - y_3_mean
    y_3 = demean(y_3)
    beta , y_3_hat , var , var_beta = OLS(X,y_3)
    res_3 = y_3_hat - y_3
    w_3 = res_3**2
    w_3_diag = w_3.copy()
    for i in range(no_state):
        w_3_diag[i::no_state] =  np.sqrt(1/(w_3[i::no_state].mean()))
    Omega_inv = np.diagflat(w_3_diag)
    X_3 = Omega_inv @ X
    y_3g = Omega_inv @ y_3
    res_3g = Omega_inv @ res_3
    # Migration
    y_4 =  d_mu_gr[
        (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_4_mean,var,var_beta = OLS(time_fixed_effects,y_4)
    y_4 = y_4 - y_4_mean
    y_4 = demean(y_4)
    beta , y_4_hat , var , var_beta = OLS(X,y_4)
    res_4 = y_4_hat - y_4
    w_4 = res_4**2
    w_4_diag = w_4.copy()
    for i in range(no_state):
        w_4_diag[i::no_state] =  np.sqrt(1/(w_4[i::no_state].mean()))
    Omega_inv = np.diagflat(w_4_diag)
    X_4 = Omega_inv @ X
    y_4g = Omega_inv @ y_4
    res_4g = Omega_inv @ res_4
    # GDP_star
    y_5 =  d_pe_gsp[
        (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_5_mean,var,var_beta = OLS(time_fixed_effects,y_5)
    y_5 = y_5 - y_5_mean
    y_5 = demean(y_5)
    beta , y_5_hat , var , var_beta = OLS(X,y_5)
    res_5 = y_5_hat - y_5
    w_5 = res_5**2
    w_5_diag = w_5.copy()
    for i in range(no_state):
        w_5_diag[i::no_state] =  np.sqrt(1/(w_5[i::no_state].mean()))
    Omega_inv = np.diagflat(w_5_diag)
    X_5 = Omega_inv @ X
    y_5g = Omega_inv @ y_5
    res_5g = Omega_inv @ res_5
    #FGLS
    X_gls = sp.linalg.block_diag(X_1,X_2,X_3,X_5)
    y_stack = np.concatenate((np.concatenate((np.concatenate((y_1g,y_2g),axis =0),y_3g),axis =0),y_5g),axis =0)
    Omega_sur = np.ones((4,4))
    Omega_sur[0,1] = np.mean(res_1g * res_2g) 
    Omega_sur[0,2] = np.mean(res_1g * res_3g)
    Omega_sur[0,3] = np.mean(res_1g * res_5g)
    Omega_sur[3,0] = Omega_sur[0,3]
    Omega_sur[2,0] = Omega_sur[0,2]
    Omega_sur[1,0] = Omega_sur[0,1]
    
    
    Omega_sur[1,2] = np.mean(res_3g * res_2g) 
    Omega_sur[1,3] = np.mean(res_5g * res_2g) 

    Omega_sur[2,1] = Omega_sur[1,2]
    Omega_sur[3,1] = Omega_sur[1,3]

    
    Omega_sur[2,3] = np.mean(res_5g * res_3g) 

    Omega_sur[3,2] = Omega_sur[2,3]

    

    Omega_sur_inv = sp.linalg.inv(Omega_sur)
    Omega_inv = np.kron(Omega_sur_inv, np.eye(time_length  * no_state,time_length  * no_state))
    beta , y_3_hat , var , var_beta = OLS(X_gls,y_stack,Omega_inv)
    eta = np.zeros((5,4))
    eta[0,3] = -1
    eta[1,3] = 1
    eta[1,0] = -1
    eta[2,0] = 1
    eta[2,1] = -1
    eta[3,1] = 1
    eta[3,2] = -1
    eta[4,2] = 1
    std_eta = np.sqrt(np.diag(eta @ var_beta @ eta.T))
    eta = eta @ beta
    eta[0] = eta[0] +1
    return eta,std_eta

In [ ]:
time_horizon_gls4(18,26)

In [ ]:
time_horizon_gls4(0,45)

In [ ]:
time_horizon_gls4(0,20)

In [ ]:
time_length = 30
time_horizon_gls4(0,45)

In [ ]:
start_period_grid = 45 - time_length
smoothing_grid = np.ones((5,start_period_grid))
time_grid = np.ones((start_period_grid,1))
for i in range(start_period_grid):
    time_grid[i,0] = i + 1970 + int(time_length/2)
    coeff,var = time_horizon_gls4(i,time_length)
    smoothing_grid[:,i] = coeff[:,0]

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["m","b", "g", "r", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Migration","Capital","Federal","Credit","Unsmoothed")
for i in np.arange(0, data_shape[0]):
    ax.bar(time_grid[:,0], data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
ax.legend(loc="best",fontsize=10)
fig.savefig('region.png', dpi=100)
plt.show()

In [ ]:
def time_horizon_gls5(start_period,time_length):
    def OLS(X,y,Omega_inv = np.eye(time_length  * no_state,time_length  * no_state)):
        beta = np.linalg.solve(X.T @ Omega_inv @  X, X.T @ Omega_inv  @ y)
        y_hat = X @ beta
        var = np.sum((y_hat - y)**2)/(time_length  * no_state - 1 )
        var_beta = var *  np.linalg.inv(X.T @ X)
        return beta , y_hat , var , var_beta
    time_fixed_effects = np.kron(np.eye(time_length,time_length),np.ones((no_state,1)))
    shape_vec = pe_cons[no_state:].shape[0]
    d_pe_cons = np.reshape(pe_cons[no_state:] - pe_cons[:-no_state],(shape_vec,1))
    d_pe_gsp = np.reshape(pe_gsp[no_state:] - pe_gsp[:-no_state],(shape_vec,1))
    d_pe_gsp_star = np.reshape(pe_gsp_star[no_state:] - pe_gsp_star[:-no_state],(shape_vec,1))
    d_pe_income = np.reshape(pe_income[no_state:] - pe_income[:-no_state],(shape_vec,1))
    d_pe_disp_income = np.reshape(pe_disp_income[no_state:] - pe_disp_income[:-no_state],(shape_vec,1))
    d_mu_gr = np.reshape(mu_gr[no_state:] - mu_gr[:-no_state],(shape_vec,1))
    
    d_pe_cons = demean(d_pe_cons)
    d_pe_gsp = demean(d_pe_gsp)
    d_pe_gsp_star = demean(d_pe_gsp_star)
    d_pe_income = demean(d_pe_income)
    d_pe_disp_income = demean(d_pe_disp_income)
    d_mu_gr = demean(d_mu_gr)
    
    y_1 =  d_pe_income[(start_period * no_state):((time_length + start_period)  * no_state)]
    X = d_pe_gsp_star[(start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,X_mean,var,var_beta = OLS(time_fixed_effects,X)
    X = X - X_mean
    X = demean(X)
    
    beta_mean,y_1_mean,var,var_beta = OLS(time_fixed_effects,y_1)
    y_1 = y_1 - y_1_mean
    y_1 = demean(y_1)
    beta , y_1_hat , var , var_beta = OLS(X,y_1)
    res_1 = y_1_hat - y_1
    w_1 = res_1**2
    w_1_diag = w_1.copy()
    for i in range(no_state):
        w_1_diag[i::no_state] =  np.sqrt(1/(w_1[i::no_state].mean()))
    Omega_inv = np.diagflat(w_1_diag)
    X_1 = Omega_inv @ X
    y_1g = Omega_inv @ y_1
    res_1g = Omega_inv @ res_1
    #Federal redistribution
    y_2 =  d_pe_disp_income[
       (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_2_mean,var,var_beta = OLS(time_fixed_effects,y_2)
    y_2 = y_2 - y_2_mean
    y_2 = demean(y_2)
    beta , y_2_hat , var , var_beta = OLS(X,y_2)
    res_2 = y_2_hat - y_2
    w_2 = res_2**2
    w_2_diag = w_2.copy()
    for i in range(no_state):
        w_2_diag[i::no_state] =  np.sqrt(1/(w_2[i::no_state].mean()))
    Omega_inv = np.diagflat(w_2_diag)
    X_2 = Omega_inv @ X
    y_2g = Omega_inv @ y_2
    res_2g = Omega_inv @ res_2
    #Credit market
    y_3 =  d_pe_cons[
        (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_3_mean,var,var_beta = OLS(time_fixed_effects,y_3)
    y_3 = y_3 - y_3_mean
    y_3 = demean(y_3)
    beta , y_3_hat , var , var_beta = OLS(X,y_3)
    res_3 = y_3_hat - y_3
    w_3 = res_3**2
    w_3_diag = w_3.copy()
    for i in range(no_state):
        w_3_diag[i::no_state] =  np.sqrt(1/(w_3[i::no_state].mean()))
    Omega_inv = np.diagflat(w_3_diag)
    X_3 = Omega_inv @ X
    y_3g = Omega_inv @ y_3
    res_3g = Omega_inv @ res_3
    # Migration
    y_4 =  d_mu_gr[
        (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_4_mean,var,var_beta = OLS(time_fixed_effects,y_4)
    y_4 = y_4 - y_4_mean
    y_4 = demean(y_4)
    beta , y_4_hat , var , var_beta = OLS(X,y_4)
    res_4 = y_4_hat - y_4
    w_4 = res_4**2
    w_4_diag = w_4.copy()
    for i in range(no_state):
        w_4_diag[i::no_state] =  np.sqrt(1/(w_4[i::no_state].mean()))
    Omega_inv = np.diagflat(w_4_diag)
    X_4 = Omega_inv @ X
    y_4g = Omega_inv @ y_4
    res_4g = Omega_inv @ res_4
    # GDP_star
    y_5 =  d_pe_gsp[
        (start_period * no_state):((time_length + start_period)  * no_state)]
    beta_mean,y_5_mean,var,var_beta = OLS(time_fixed_effects,y_5)
    y_5 = y_5 - y_5_mean
    y_5 = demean(y_5)
    beta , y_5_hat , var , var_beta = OLS(X,y_5)
    res_5 = y_5_hat - y_5
    w_5 = res_5**2
    w_5_diag = w_5.copy()
    for i in range(no_state):
        w_5_diag[i::no_state] =  np.sqrt(1/(w_5[i::no_state].mean()))
    Omega_inv = np.diagflat(w_5_diag)
    X_5 = Omega_inv @ X
    y_5g = Omega_inv @ y_5
    res_5g = Omega_inv @ res_5
    #FGLS
    X_gls = sp.linalg.block_diag(X_1,X_2,X_3,X_5)
    y_stack = np.concatenate((np.concatenate((np.concatenate((y_1g,y_2g),axis =0),y_3g),axis =0),y_5g),axis =0)
    Omega_sur = np.ones((4,4))
    Omega_sur[0,1] = np.mean(res_1g * res_2g) 
    Omega_sur[0,2] = np.mean(res_1g * res_3g)
    Omega_sur[0,3] = np.mean(res_1g * res_5g)
    Omega_sur[3,0] = Omega_sur[0,3]
    Omega_sur[2,0] = Omega_sur[0,2]
    Omega_sur[1,0] = Omega_sur[0,1]
    
    
    Omega_sur[1,2] = np.mean(res_3g * res_2g) 
    Omega_sur[1,3] = np.mean(res_5g * res_2g) 

    Omega_sur[2,1] = Omega_sur[1,2]
    Omega_sur[3,1] = Omega_sur[1,3]

    
    Omega_sur[2,3] = np.mean(res_5g * res_3g) 

    Omega_sur[3,2] = Omega_sur[2,3]

    

    Omega_sur_inv = sp.linalg.inv(Omega_sur)
    Omega_inv = np.kron(Omega_sur_inv, np.eye(time_length  * no_state,time_length  * no_state))
    beta , y_3_hat , var , var_beta = OLS(X_gls,y_stack,Omega_inv)
    eta = np.zeros((4,4))
    eta[0,3] = -1
    eta[1,3] = 1
    eta[1,0] = -1
    eta[2,0] = 1
    eta[2,1] = -1
    eta[3,1] = 1
    eta[3,2] = 0
    #eta[4,2] = 1
    std_eta = np.sqrt(np.diag(eta @ var_beta @ eta.T))
    eta = eta @ beta
    eta[0] = eta[0] +1
    return eta,std_eta

In [ ]:
time_horizon_gls5(19,26)[0]

In [ ]:
smoothing_grid = np.array([ 0.04482439,  0.08561039,  0.0319404 ,
  0.46531857,  0.1549013,   0.0210731 ,
  0.12940698 , 0.19835615  ,0.01708878,
  0.15337855,  0.13644383 , 0.05553481,
  0.2070715 ,  0.42468834  ,0.87436292])
smoothing_grid = smoothing_grid.reshape(5,3)
vertical_grid = ['US States','US Regions','EU members']

In [ ]:
#smoothing_grid[:,0] = time_horizon_gls4(19,26)[0][:,0]

In [ ]:
#smoothing_grid[:,1] = time_horizon_gls4(19,26)[0][:,0]

In [ ]:
#smoothing_grid[:,2] = time_horizon_gls4(18,25)[0][:,0]

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["purple","gray", "g", "y", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Migration","Capital","Federal","Credit","Unsmoothed")
const =np.zeros((3,1))
for i in np.arange(0, data_shape[0]):
    rects1 = ax.bar(np.linspace(1,3,3), data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
    
    for j in range(3):
        h1 = rects1[j].get_height()
        x = h1-0.04-0.02*i +1.01*const[j]
        text1 = '' + str(round(100 * h1,1)) + ' %'
        if j == 2:
            if (i ==4):
                x = h1-0.4-0.02*i +1.01*const[j]
                plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=24, fontweight="bold")
            elif(i == 3):
                x = h1-0.06+0.03*i +0.2*const[j]
                plt.text(rects1[j].get_x() + rects1[j].get_width() / 2. + 0.47,  x, text1 , ha="center",  color="black", fontsize=14, fontweight="bold")
            else:
                x = h1-0.03+0.03*i +0.2*const[j]
                plt.text(rects1[j].get_x() + rects1[j].get_width() / 2. + 0.47,  x, text1 , ha="center",  color="black", fontsize=14, fontweight="bold")
        else:
            plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=24, fontweight="bold")
        
        const[j] =  h1 + const[j]
plt.xticks(np.linspace(1,3,3), vertical_grid, fontweight='bold',fontsize=24)

ax.legend(loc="best",fontsize=24)
fig.savefig('comparison.png', dpi=100)
tikz_save('comparison.tex', figureheight='\\figureheight',
    figurewidth='\\figurewidth')
plt.show()

In [ ]:
get_tikz_code?


In [ ]:
smoothing_grid = np.array([ 0.08561039, 0.0864, 
   0.1549013, 0.1548 ,
   0.19835615  , 0.1963,
   0.42468834 + 0.13644383 , 0.5625])
smoothing_grid = smoothing_grid.reshape(4,2)
vertical_grid = ['US Regions','Baseline']#,'Low migration','Costly adjustment']

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["purple","gray", "g", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Migration","Capital","Federal","Unsmoothed")
const =np.zeros((smoothing_grid.shape[1],1))
for i in np.arange(0, data_shape[0]):
    rects1 = ax.bar(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
    
    for j in range(smoothing_grid.shape[1]):
        h1 = rects1[j].get_height()
        x = h1-0.04-0.02*i +1.01*const[j]
        text1 = '' + str(round(100 * h1,1)) + ' %'
        plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=24, fontweight="bold")
        
        const[j] =  h1 + const[j]
plt.xticks(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), vertical_grid, fontweight='bold',fontsize=24)

ax.legend(loc="center right",fontsize=24)
fig.savefig('comparison2.png', dpi=100)
tikz_save('comparison2.tikz', figureheight='\\figureheight',
    figurewidth='\\figurewidth')
plt.show()

In [ ]:
smoothing_grid = np.array([ 0.0864, 0.0808,
    0.1548 ,0.1711,
    0.1963,0,
    0.5625,0.7482])
smoothing_grid = smoothing_grid.reshape(4,2)
vertical_grid = ['Baseline','No Federal redistribution']#,'Low migration','Costly adjustment']

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["purple","gray", "g", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Migration","Capital","Federal","Unsmoothed")
const =np.zeros((smoothing_grid.shape[1],1))
for i in np.arange(0, data_shape[0]):
    rects1 = ax.bar(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
    
    for j in range(smoothing_grid.shape[1]):
        h1 = rects1[j].get_height()
        if h1 != 0:
            x = h1-0.04-0.02*i +1.01*const[j]
            text1 = '' + str(round(100 * h1,1)) + ' %'
            plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=24, fontweight="bold")

            const[j] =  h1 + const[j]
plt.xticks(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), vertical_grid, fontweight='bold',fontsize=24)

ax.legend(loc="center right",fontsize=24)
fig.savefig('comparison_no_federal.png', dpi=100)
tikz_save('comparison2.tikz', figureheight='\\figureheight',
    figurewidth='\\figurewidth')
plt.show()

In [ ]:
smoothing_grid = np.array([ 0.0864, 0.0190,
    0.1548 ,0.1247,
    0.1963,0.2124,
    0.5625,0.6439])
smoothing_grid = smoothing_grid.reshape(4,2)
vertical_grid = ['Baseline','Low migration']#,'Low migration','Costly adjustment']

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["purple","gray", "g", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Migration","Capital","Federal","Unsmoothed")
const =np.zeros((smoothing_grid.shape[1],1))
for i in np.arange(0, data_shape[0]):
    rects1 = ax.bar(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
    
    for j in range(smoothing_grid.shape[1]):
        h1 = rects1[j].get_height()
        if h1 > 0.02:
            x = h1-0.04-0.02*i +1.01*const[j]
            text1 = '' + str(round(100 * h1,1)) + ' %'
            plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=24, fontweight="bold")

            const[j] =  h1 + const[j]
        else:
            x = h1-0.01-0.02*i +1.01*const[j]
            text1 = '' + str(round(100 * h1,2)) + ' %'
            plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=16, fontweight="bold")

            const[j] =  h1 + const[j]
plt.xticks(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), vertical_grid, fontweight='bold',fontsize=24)

ax.legend(loc="center right",fontsize=24)
fig.savefig('comparison_low_migr.png', dpi=100)
tikz_save('comparison2.tikz', figureheight='\\figureheight',
    figurewidth='\\figurewidth')
plt.show()

In [ ]:
smoothing_grid = np.array([ 0.0864, 0.0795,
    0.1548 ,0.0869,
    0.1963,0.1949,
    0.5625,0.6388])
smoothing_grid = smoothing_grid.reshape(4,2)
vertical_grid = ['Baseline','Costly adjustment']#,'Low migration','Costly adjustment']

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["purple","gray", "g", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Migration","Capital","Federal","Unsmoothed")
const =np.zeros((smoothing_grid.shape[1],1))
for i in np.arange(0, data_shape[0]):
    rects1 = ax.bar(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
    
    for j in range(smoothing_grid.shape[1]):
        h1 = rects1[j].get_height()
        if h1 > 0.02:
            x = h1-0.04-0.02*i +1.01*const[j]
            text1 = '' + str(round(100 * h1,1)) + ' %'
            plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=24, fontweight="bold")

            const[j] =  h1 + const[j]
        else:
            x = h1-0.01-0.02*i +1.01*const[j]
            text1 = '' + str(round(100 * h1,1)) + ' %'
            plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=16, fontweight="bold")

            const[j] =  h1 + const[j]
plt.xticks(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), vertical_grid, fontweight='bold',fontsize=24)

ax.legend(loc="center right",fontsize=24)
fig.savefig('comparison_high_adj.png', dpi=100)
tikz_save('comparison2.tikz', figureheight='\\figureheight',
    figurewidth='\\figurewidth')
plt.show()

In [ ]:
smoothing_grid = np.array([  0, 0,0.0006,0.0006,
    0.0655 ,0.0747,0.0686,0.3526,
    0.0,0.2537,0,0,
    0.9347,0.6718,0.9318,0.6468])
smoothing_grid = smoothing_grid.reshape(4,4)
vertical_grid = ['Baseline','Federal government','Higher migration','Better adjustment']#,'Low migration','Costly adjustment']

In [ ]:
data = smoothing_grid
data_shape = np.shape(smoothing_grid)

# Take negative and positive data apart and cumulate
def get_cumulated_array(data, **kwargs):
    cum = data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(data, min=0)
cumulated_data_neg = get_cumulated_array(data, max=0)

# Re-merge negative and positive data.
row_mask = (data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["purple","gray", "g", "c"]

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(111)
labels = ("Migration","Capital","Federal","Unsmoothed")
const =np.zeros((smoothing_grid.shape[1],1))
for i in np.arange(0, data_shape[0]):
    rects1 = ax.bar(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), data[i], label=labels[i], bottom=data_stack[i], color=cols[i],)
    
    for j in range(smoothing_grid.shape[1]):
        h1 = rects1[j].get_height()
        if h1 > 0.02:
            x = h1-0.04-0.02*i +1.01*const[j]
            text1 = '' + str(round(100 * h1,1)) + ' %'
            plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=24, fontweight="bold")

            const[j] =  h1 + const[j]
        else:
            x = h1-0.01-0.02*i +1.01*const[j]
            text1 = '' + str(round(100 * h1,1)) + ' %'
            #plt.text(rects1[j].get_x() + rects1[j].get_width() / 2.,  x, text1 , ha="center",  color="white", fontsize=16, fontweight="bold")

            const[j] =  h1 + const[j]
plt.xticks(np.linspace(1,smoothing_grid.shape[1],smoothing_grid.shape[1]), vertical_grid, fontweight='bold',fontsize=16)

ax.legend(loc="center right",fontsize=24)
fig.savefig('comparison3.png', dpi=100)
#tikz_save('comparison3.tikz', figureheight='\\figureheight',
#    figurewidth='\\figurewidth')
plt.show()